# データ操作

このセクションでは、3つの主要なトピックについて説明します。

1. ワイド(太)形式からロング(細長)形式へのデータ形状変更
2. ロング(細長)形式からワイド(太)形式へのデータ形状変更
3. データセットの結合

データ形状を変更するために、次の2つの方法について説明します。

* PROC TRANSPOSE
* 配列を使ったデータステップ

2番目の一般的な方法を理解するには、まずいくつかのSASプログラミングのキーワードと構造について学ぶ必要があります。

* OUTPUT文とRETAIN文
* SASでのループ
* SAS配列
* FIRST.変数とLAST.変数

## OUTPUT文とRETAIN文

データステップを処理する際、SASは2つのデフォルトの手順に従います。

1. データステップの開始時にSASがDATA文を読み取ると、INPUT文または代入文によって割り当てられた変数にはSASがミッシング値を設定します。(SUM文によって作成された変数、またはSET文やMERGE文によってSASデータセットから値が来た場合は、SASは変数をミッシング値にリセットしません。)
2. データステップの最後で、SASは変数の値をプログラムデータベクターから作成中のSASデータセットに出力します。

このレッスンでは、OUTPUT文とRETAIN文を使ってこれらのデフォルトの動作を変更する方法を学びます。

* **OUTPUT**文を使うと、観測値をいつ、どのデータセットに書き出すかを制御できます。
* **RETAIN**文は、データステップで作成された変数が次の観測値に渡されるように指示するので、各反復の開始時にミッシング値にリセットされることがありません。

### OUTPUT文

OUTPUT文を使うと、デフォルトの動作を上書きして、OUTPUT文が処理されたタイミングで現在の観測値を出力するようSASに指示できます。OUTPUT文の形式は次のようになります。

`OUTPUT dataset1 dataset2 ... datasetn;`;

ここで、dataset1、dataset2、...、datasetnはデータセット名で、いくつでも指定できます。データセット名を指定しないでOUTPUT文を使うと、SASはDATA文で名前が付けられたすべてのデータセットに現在の観測値を書き出します。OUTPUT文に現れるデータセット名は、すべてDATA文にも現れる必要があります。

OUTPUT文は非常に強力で、次のようなことができます。

* 複数のデータセットに観測値を書き出すこと
* 特定の条件に基づいてデータセットへの観測値の出力を制御すること
* OUTPUT文とRETAIN文、BY グループ処理、LAST.変数を組み合わせることで、データセットを転置すること

このセクションの残りの部分では、OUTPUT文を正しく使う方法を示す例を見ていきます。ここでは、ICDBスタディのログデータセット(icdblog.sas7bdatはコースウェブサイトから入手可能)の一部を使って作業します。

<!-- 
# Data Manipulation

In this section, we will cover three main topics

1. Reshaping data frome wide (fat) to long (tall) formats
2. Reshaping data from long (tall) to wide (fat) formats
3. Merging datasets

To reshape datasets, we will cover two methods

* PROC TRANSPOSE
* Using a DATA step with arrays

In order to understand the second more general method, we will first need to learn about a few SAS programming keywords and structures, such as 

* The OUTPUT and RETAIN statements
* Loops in SAS
* SAS Arrays
* FIRST. and LAST. SAS variables

## The OUTPUT and RETAIN Statements

When processing any DATA step, SAS follows two default procedures:

1. When SAS reads the DATA statement at the beginning of each iteration of the DATA step, SAS places missing values in the program data vector for variables that were assigned by either an INPUT statement or an assignment statement within the DATA step. (SAS does not reset variables to missing if they were created by a SUM statement, or if the values came from a SAS data set via a SET or MERGE statement.)
2. At the end of the DATA step after completing an iteration of the DATA step, SAS outputs the values of the variables in the program data vector to the SAS data set being created.

In this lesson, we'll learn how to modify these default processes by using the OUTPUT and RETAIN statements:

* The **OUTPUT** statement allows you to control when and to which data set you want an observation written.
* The **RETAIN** statement causes a variable created in the DATA step to retain its value from the current observation into the next observation rather than it being set to missing at the beginning of each iteration of the DATA step.

### The OUTPUT Statement

An OUTPUT statement overrides the default process by telling SAS to output the current observation when the OUTPUT statement is processed — not at the end of the DATA step. The OUTPUT statement takes the form:

`OUTPUT dataset1 dataset2 ... datasetn;`;

where you may name as few or as many data sets as you like. If you use an OUTPUT statement without specifying a data set name, SAS writes the current observation to each of the data sets named in the DATA step. Any data set name appearing in the OUTPUT statement must also appear in the DATA statement.

The OUTPUT statement is pretty powerful in that, among other things, it gives us a way:

* to write observations to multiple data sets
* to control output of observations to data sets based on certain conditions
* to transpose datasets using the OUTPUT statement in conjunction with the RETAIN statement, BY group processing and the LAST.variable statement

Throughout the rest of this section, we'll look at examples that illustrate how to use OUTPUT statements correctly. We'll work with the following subset of the ICDB Study's log data set (see the course website for icdblog.sas7bdat):
-->

In [1]:
LIBNAME PHC6089 "/folders/myfolders/SAS_Notes/data";

PROC PRINT data = phc6089.icdblog (obs=5);
RUN;

SAS Connection established. Subprocess id is 2262



Obs,SUBJ,V_TYPE,V_DATE,FORM
1,210006,12,05/06/94,cmed
2,210006,12,05/06/94,diet
3,210006,12,05/06/94,med
4,210006,12,05/06/94,phytrt
5,210006,12,05/06/94,purg


ログデータセットには、次の4つの変数が含まれています。

* `subj`: 被験者の識別番号
* `v_type`: 診療所の来院回数で、最初の来院から何ヶ月経過したかを表す
* `v_date`: 診療所の来院日
* `form`: 被験者の来院時に記入されたデータフォームのコード

ログデータセットは、全国的な臨床研究で典型的に見られるデータセットの一種です。国内の複数の拠点で集められたデータが、データ調整センター(DCC)に集められます。DCCでデータフォームを追跡管理するのは大変な作業です。例えば、ICDBスタディではスタディ期間中に68,000を超えるデータフォームが集められました。

DCCに届いたデータフォームは、データベースに記録され、その後の処理が追跡されます。実際のログデータベースには、ここで扱う4つの変数以外にも、データがデータベースに入力された日付、入力者、データの検証日、検証者など、多くの変数が含まれています。ここでは単純化するために、これらの4つの変数のみを扱います。

例
この例では、OUTPUT文を使ってSASにデータセットへの観測値の書き出し条件を指示しています。具体的には、次のプログラムではOUTPUT文を使って、icdblogデータセットの被験者識別番号が特定の条件を満たすかどうかに応じて、s210006、s310032、s410010の3つのSASデータセットを作成しています。



<!-- 
As you can see, this log data set contains four variables:

* `subj`: the subject's identification number
* `v_type`: the type of clinic visit, which means the number of months since the subject was first seen in the clinic
* `v_date`: the date of the clinic visit
* `form`: codes that indicate the data forms that were completed during the subject's clinic visit

The log data set is a rather typical data set that arises from large national clinical studies in which there are a number of sites around the country where data are collected. Typically, the clinical sites collect the data on data forms and then "ship" the data forms either electronically or by mail to a centralized location called a Data Coordinating Center (DCC). As you can well imagine, keeping track of the data forms at the DCC is a monumental task. For the ICDB Study, for example, the DCC received more than 68,000 data forms over the course of the study.

In order to keep track of the data forms that arrive at the DCC, they are "logged" into a data base and subsequently tracked as they are processed at the DCC. In reality, a log data base will contain many more variables than we have in our subset, such as dates the data on the forms were entered into the data base, who entered the data, the dates the entered data were verified, who verified the data, and so on. To keep our life simple, we'll just use the four variables described above.

### Example

This example uses the OUTPUT statement to tell SAS to write observations to data sets based on certain conditions. Specifically, the following program uses the OUTPUT statement to create three SAS data sets — s210006, s310032, and s410010 — based on whether the subject identification numbers in the icdblog data set meet a certain condition:  
-->

In [11]:
LIBNAME PHC6089 "/folders/myfolders/SAS_Notes/data";

DATA s210006 s310032 s410010;
    set phc6089.icdblog;
        if (subj = 210006) then output s210006;
    else if (subj = 310032) then output s310032;
    else if (subj = 410010) then output s410010;
RUN;
 
PROC PRINT data = s210006 (obs=5) NOOBS;
    title 'The s210006 data set';
RUN;
 
PROC PRINT data = s310032 (obs=5) NOOBS;
    title 'The s310032 data set';
RUN;
 
PROC PRINT data = s410010 (obs=5) NOOBS;
    title 'The s410010 data set';
RUN;

SUBJ,V_TYPE,V_DATE,FORM
210006,12,05/06/94,cmed
210006,12,05/06/94,diet
210006,12,05/06/94,med
210006,12,05/06/94,phytrt
210006,12,05/06/94,purg
SUBJ,V_TYPE,V_DATE,FORM
310032,24,09/19/95,backf
310032,24,09/19/95,cmed
310032,24,09/19/95,diet
310032,24,09/19/95,med



DATA文には、s210006、s310032、s410010の3つのデータセット名が含まれています。これはSASに対して、これらの名前のデータセットを作成するように指示しています。SET文は、stat481.icdblogという永続的なデータセットから観測値を読み込むようSASに指示しています。次にIF-THEN-ELSE文とOUTPUT文が効果を発揮します。最初のIF-THENは、被験者210006の観測値をs210006データセットに出力するようSASに指示しています。2番目のIF-THENは、被験者310032の観測値をs310032データセットに出力するようSASに指示しています。3番目のIF-THEN文は、被験者410010の観測値をs410010データセットに出力するようSASに指示しています。OUTPUT文にデータセット名を指定していないと、SASはエラーを出します。
PRINT手順は、新しく作成された3つのデータセットを出力するようSASに指示しています。最後のPRINT手順にはDATA=オプションがありません。DATA文で複数のデータセット名を指定すると、最後の名前が最新に作成されたデータセットになるので、後続の手順ではそのデータセットが自動的に使用されるからです。したがって、最後のPRINT手順はs410010データセットを自動的に出力します。
ここでOUTPUT文を使うと、データステップの最後での観測値の自動出力が抑制されます。したがって、データステップでOUTPUT文を使う場合は、そのステップのすべての出力をOUTPUT文でプログラミングする必要があります。次のSASプログラムは、すべての観測値の出力を指示しなかった場合の動作を示しています。

<!-- 
As you can see, the DATA statement contains three data set names — s210006, s310032, and s410010. That tells SAS that we want to create three data sets with the given names. The SET statement, of course, tells SAS to read observations from the permanent data set called stat481.icdblog. Then comes the IF-THEN-ELSE and OUTPUT statements that make it all work. The first IF-THEN tells SAS to output any observations pertaining to subject 210006 to the s210006 data set; the second IF-THEN tells SAS to output any observations pertaining to subject 310032 to the s310032 data set; and, the third IF-THEN statement tells SAS to output any observations pertaining to subject 410010 to the s410010 data set. SAS will hiccup if you have a data set name that appears in an OUTPUT statement without it also appearing in the DATA statement.  
The PRINT procedures, of course, tell SAS to print the three newly created data sets. Note that the last PRINT procedure does not have a DATA= option. That's because when you name more than one data set in a single DATA statement, the last name on the DATA statement is the most recently created data set, and the one that subsequent procedures use by default. Therefore, the last PRINT procedure will print the s410010 data set by default.  
Note that the IF-THEN-ELSE construct used here in conjunction with the OUTPUT statement is comparable to attaching the WHERE= option to each of the data sets appearing in the DATA statement.  
Before running the code be sure that you have saved the icdblog dataset and changed the LIBNAME statement to the folder where you saved it.  
-->

### Example

Using an OUTPUT statement suppresses the automatic output of observations at the end of the DATA step. Therefore, if you plan to use any OUTPUT statements in a DATA step, you must use OUTPUT statements to program all of the output for that step. The following SAS program illustrates what happens if you fail to direct all of the observations to output:  

In [13]:
DATA subj210006 subj310032;
    set phc6089.icdblog;
    if (subj = 210006) then output subj210006;
RUN;
 
PROC PRINT data = subj210006 NOOBS;
    title 'The subj210006 data set';
RUN;

SUBJ,V_TYPE,V_DATE,FORM
210006,12,05/06/94,cmed
210006,12,05/06/94,diet
210006,12,05/06/94,med
210006,12,05/06/94,phytrt
210006,12,05/06/94,purg
210006,12,05/06/94,qul
210006,12,05/06/94,sympts
210006,12,05/06/94,urn
210006,12,05/06/94,void


In [14]:
PROC PRINT data = subj310032 NOOBS;
    title 'The subj310032 data set';
RUN;

DATA文には2つのデータセット名subj210006とsubj310032が含まれているので、SASにはこの2つのデータセットを作成するよう指示しています。しかし、IF文にはsubj210006データセットへの出力を指示するOUTPUT文があるものの、subj310032データセットへの出力を指示するOUTPUT文がありません。プログラムを実行すると、subj210006データセットには被験者210006のデータが入っているのに対し、subj310032データセットには観測値が0個入っていることがわかります。

<!-- 
The DATA statement contains two data set names, subj210006 and subj310032, telling SAS that we intend to create two data sets. However, as you can see, the IF statement contains an OUTPUT statement that directs output to the subj210006 data set, but no OUTPUT statement directs output to the subj310032 data set. Launch and run the SAS program to convince yourself that the subj210006 data set contains data for subject 210006, while the subj310032 data set contains 0 observations. You should see a message in the log window like the one shown above as well as see that no output for the subj310032 data set appears in the output window.  
-->

例 
データステップ内でOUTPUT文を使い、新しい変数を割り当て文で作成する場合、その割り当て文をOUTPUT文の前に置く必要があります。そうしないと、SASはすでに観測値をSASデータセットに書き出した後になって、新しい変数の値が計算されるので、その変数の値はミッシング値になってしまいます。次のSASプログラムは、OUTPUT文があるときに、currentとdays_vis変数をデータセットに書き込もうとすると、ミッシング値になってしまう例を示しています。



<!-- 
### Example

If you use an assignment statement to create a new variable in a DATA step in the presence of OUTPUT statements, you have to make sure that you place the assignment statement **before** the OUTPUT statements. Otherwise, SAS will have already written the observation to the SAS data set, and the newly created variable will be set to missing. The following SAS program illustrates an example of how two variables, current and days_vis, get set to missing in the output data sets because their values get calculated after SAS has already written the observation to the SAS data set:  
-->

In [15]:
DATA subj210006 subj310032 subj410010;
    set phc6089.icdblog;
        if (subj = 210006) then output subj210006;
    else if (subj = 310032) then output subj310032;
    else if (subj = 410010) then output subj410010;
    current = today();
    days_vis = current - v_date;
    format current mmddyy8.;
RUN;
 
PROC PRINT data = subj310032 NOOBS;
    title 'The subj310032 data set';
RUN;

SUBJ,V_TYPE,V_DATE,FORM,current,days_vis
310032,24,09/19/95,backf,.,.
310032,24,09/19/95,cmed,.,.
310032,24,09/19/95,diet,.,.
310032,24,09/19/95,med,.,.
310032,24,09/19/95,medhxf,.,.
310032,24,09/19/95,phs,.,.
310032,24,09/19/95,phytrt,.,.
310032,24,09/19/95,preg,.,.
310032,24,09/19/95,purg,.,.
310032,24,09/19/95,qul,.,.


この例で重要なのは、currentとdays_vis代入文がIF-THEN-ELSE文とOUTPUT文の後に来ていることです。つまり、各観測値がいずれかの出力データセットに書き出された後で、currentとdays_visの値が計算されることになります。データステップで作成された変数は、反復のたびにミッシング値にリセットされるので、currentとdays_visの値はすべての観測値についてミッシング値のままになります。

次のSASプログラムは、OUTPUT文がある場合の割り当て文の正しいコードです。


<!-- 
The main thing to note in this program is that the current and days_vis assignment statements appear after the IF-THEN-ELSE and OUTPUT statements. That means that each observation will be written to one of the three output data sets before the current and days_vis values are even calculated. Because SAS sets variables created in the DATA step to missing at the beginning of each iteration of the DATA step, the values of current and days_vis will remain missing for each observation.  
By the way, the today( ) function, which is assigned to the variable current, creates a date variable containing today's date. Therefore, the variable days_vis is meant to contain the number of days since the subject's recorded visit v_date. However, as described above, the values of current and days_vis get set to missing. Launch and run the SAS program to convince yourself that the current and days_vis variables in the subj310032 data set contain only missing values. If we were to print the subj210006 and subj410020 data sets, we would see the same thing.  
The following SAS program illustrates the corrected code for the previous DATA step, that is, for creating new variables with assignment statements in the presence of OUTPUT statements:  
-->

In [16]:
DATA subj210006 subj310032 subj410010;
    set phc6089.icdblog;
    current = today();
    days_vis = current - v_date;
    format current mmddyy8.;
        if (subj = 210006) then output subj210006;
    else if (subj = 310032) then output subj310032;
    else if (subj = 410010) then output subj410010;
RUN;
 
PROC PRINT data = subj310032 NOOBS;
    title 'The subj310032 data set';
RUN;

SUBJ,V_TYPE,V_DATE,FORM,current,days_vis
310032,24,09/19/95,backf,09/30/20,9143
310032,24,09/19/95,cmed,09/30/20,9143
310032,24,09/19/95,diet,09/30/20,9143
310032,24,09/19/95,med,09/30/20,9143
310032,24,09/19/95,medhxf,09/30/20,9143
310032,24,09/19/95,phs,09/30/20,9143
310032,24,09/19/95,phytrt,09/30/20,9143
310032,24,09/19/95,preg,09/30/20,9143
310032,24,09/19/95,purg,09/30/20,9143
310032,24,09/19/95,qul,09/30/20,9143


(コードは省略)  

割り当て文がOUTPUT文の前にあるので、変数が正しく出力データセットに書き込まれます。つまり、current変数にはプログラムが実行された日付が入り、days_vis変数には被験者の来院日からその日までの日数が入ります。  


(コードは省略)


Now, since the assignment statements precede the OUTPUT statements, the variables are correctly written to the output data sets. That is, now the variable current contains the date in which the program was run and the variable days_vis contains the number of days since that date and the date of the subject's visit. Launch and run the SAS program to convince yourself that the current and days_vis variables are properly written to the subj310032 data set. If we were to print the subj210006 and subj410020 data sets, we would see similar results.



例
OUTPUT文をデータステップで処理した後、観測値はプログラムデータベクターに残り、さらにプログラミングを続けることができます。同じSASデータセットまたは別のデータセットに、その観測値を再び出力することもできます。次のSASプログラムは、一部の観測値を共有する異なるデータセットを作成する方法を示しています。つまり、DATA文で作成するデータセットは必ずしも相互に排他的である必要はありません。  


<!-- 
### Example

After SAS processes an OUTPUT statement within a DATA step, the observation remains in the program data vector and you can continue programming with it. You can even output the observation again to the same SAS data set or to a different one! The following SAS program illustrates how you can create different data sets with the some of the same observations. That is, the data sets created in your DATA statement do not have to be mutually exclusive:  
-->

In [17]:
DATA symptoms visitsix;
    set phc6089.icdblog;
    if form = 'sympts' then output symptoms;
    if v_type = 6 then output visitsix;
RUN;
 
PROC PRINT data = symptoms NOOBS;
    title 'The symptoms data set';
RUN;
 
PROC PRINT data = visitsix NOOBS;
    title 'The visitsix data set';
RUN;

SUBJ,V_TYPE,V_DATE,FORM
210006,12,05/06/94,sympts
310032,24,09/19/95,sympts
410010,6,05/12/94,sympts
SUBJ,V_TYPE,V_DATE,FORM
410010,6,05/12/94,cmed
410010,6,05/12/94,diet
410010,6,05/12/94,med
410010,6,05/12/94,phytrt
410010,6,05/12/94,purg
410010,6,05/12/94,qul


The DATA step creates two temporary data sets, symptoms and visitsix. The symptoms data set contains only those observations containing a form code of sympts. The visitsix data set, on the other hand, contains observations for which v_type equals 6. The observations in the two data sets are therefore not necessarily mutually exclusive. In fact, launch and run the SAS program and review the output from the PRINT procedures. Note that the observation for subject 410010 in which form = sympts is contained in both the symptoms and visitsix data sets.  

### RETAIN文

データステップの開始時にSASがDATA文を読み取ると、INPUT文または代入文で割り当てられた変数にはSASがミッシング値を設定します。RETAIN文を使うと、このデフォルトの動作を上書きできます。つまり、RETAIN文を使うと、INPUT文または代入文で値が割り当てられた変数の値が、次の反復に進むときにミッシング値にリセットされないようにSASに指示します。代わりに、SASはその値を保持します。RETAIN文の一般的な形式は次のとおりです。

`RETAIN variable1 variable2 ... variablen;`

変数名は1つでも複数でも指定できます。変数名を指定しない場合、SASはINPUT文または代入文で作成されたすべての変数の値を保持します。RETAIN文内で変数の値を初期化することもできます。例えば、次の文

`RETAIN var1 0 var2 3 a b c 'XYZ'`

では、var1には0が、var2には3が割り当てられ、a、b、cには文字列'XYZ'が割り当てられます。初期値を指定しない場合、SASは保持される変数の初期値をミッシング値に設定します。

最後に、RETAIN文は実行可能な文ではないので、データステップのどこに置いても構いません。

このレッスンの残りの部分では、次のデータステップで作成されるgradesデータセットを使って作業します。

(コードは省略)

<!-- 
### The RETAIN Statement

When SAS reads the DATA statement at the beginning of each iteration of the DATA step, SAS places missing values in the program data vector for variables that were assigned by either an INPUT statement or an assignment statement within the DATA step. A RETAIN statement effectively overrides this default. That is, a RETAIN statement tells SAS not to set variables whose values are assigned by an INPUT or assignment statement to missing when going from the current iteration of the DATA step to the next. Instead, SAS retains the values. The RETAIN statement takes the generic form:

`RETAIN variable1 variable2 ... variablen;`

You can specify as few or as many variables as you want. If you specify no variable names, then SAS retains the values of all of the variables created in an INPUT or assignment statement. You may initialize the values of variables within a RETAIN statement. For example, in the statement:

`RETAIN var1 0 var2 3 a b c 'XYZ'`

the variable var1 is assigned the value 0; the variable var2 is assigned the value 3, and the variables a, b, and c are all assigned the character value 'XYZ'. If you do not specify an initial value, SAS sets the initial value of a variable to be retained to missing.

Finally, since the RETAIN statement is not an executable statement, it can appear anywhere in the DATA step.

Throughout the remainder of the lesson, we will work with the grades data set that is created in the following DATA step:
-->

In [18]:
DATA grades;
    input idno 1-2 l_name $ 5-9 gtype $ 12-13 grade 15-17;
    cards;
10  Smith  E1  78
10  Smith  E2  82
10  Smith  E3  86
10  Smith  E4  69
10  Smith  P1  97
10  Smith  F1 160
11  Simon  E1  88
11  Simon  E2  72
11  Simon  E3  86
11  Simon  E4  99
11  Simon  P1 100
11  Simon  F1 170
12  Jones  E1  98
12  Jones  E2  92
12  Jones  E3  92
12  Jones  E4  99
12  Jones  P1  99
12  Jones  F1 185
;
RUN;
 
PROC PRINT data = grades (obs=5) NOOBS;
    title 'The grades data set';
RUN;

idno,l_name,gtype,grade
10,Smith,E1,78
10,Smith,E2,82
10,Smith,E3,86
10,Smith,E4,69
10,Smith,P1,97


gradesデータセットは、「被験者別・成績別」データセットと呼ばれるものです。つまり、被験者ごと、成績ごとに1つの観測値があります。被験者は識別番号(idno)と名字(l_name)で特定されます。このデータセットには、6種類の成績が含まれています。100点満点の4回の試験(E1、E2、E3、E4)、100点のプロジェクト(P1)、200点の期末試験(F1)です。このデータセットでは、説明を簡単にするため、4回の試験のうち最低の成績は落とすことにしましょう。プログラムを実行して、gradesデータセットを確認してください。PRINT手順の出力を見て、データが正しく読み込まれていることを確認してください。

RETAIN文の例を見る前に、SAS変数FIRST.とLAST.について見ていきましょう。

<!-- 
The grades data set is what we call a "subject- and grade-specific" data set. That is, there is one observation for each grade for each student. Students are identified by their id number (idno) and last name (l_name). The data set contains six different types of grades: exam 1 (E1), exam 2 (E2), exam 3 (E3), exam 4 (E4), each worth 100 points; one project (P1) worth 100 points; and a final exam (F1) worth 200 points. We'll suppose that the instructor agreed to drop the students' lowest exam grades (E1, E2, E3, E4) not including the final exam. Launch and run the SAS program so that we can work with the grades data set in the following examples. Review the output from the PRINT procedure to convince yourself that the data were properly read into the grades data set.

Before we look at an example using the RETAIN statement, let's look at the SAS variables FIRST. and LAST.
-->


例
次のSASプログラムは、BY文を使ってデータセットをソートし、データステップ内でSAS変数FIRST.とLAST.を取得する方法を示しています。これにより、各被験者の最初と最後の成績レコードを識別できます。 


<!-- 
### Example

The following SAS program illustrates the SAS variables FIRST. and LAST. that can be obtained when using the BY statement on a sorted dataset in a DATA step to identify the first and last grade record for each student in the dataset.  
-->

In [19]:
PROC SORT data = grades out = srt_grades;
   BY idno;
RUN;

DATA grades_first_last;
   SET srt_grades;
   BY idno;
   firstGrade = FIRST.idno;
   lastGrade = LAST.idno;
RUN;

PROC PRINT data = grades_first_last;
RUN;

Obs,idno,l_name,gtype,grade,firstGrade,lastGrade
1,10,Smith,E1,78,1,0
2,10,Smith,E2,82,0,0
3,10,Smith,E3,86,0,0
4,10,Smith,E4,69,0,0
5,10,Smith,P1,97,0,0
6,10,Smith,F1,160,0,1
7,11,Simon,E1,88,1,0
8,11,Simon,E2,72,0,0
9,11,Simon,E3,86,0,0
10,11,Simon,E4,99,0,0


(コードは省略)

変数idnoでBYグループ処理を行うため、データセットをidnoでソートする必要があります。この例ではデータセットはすでにidnoでソートされていますが、ソートが必要であることを強調するためにPROC SORTを追加しています。

SET文とBY文は、SASに対して、idnoが同じ観測値をグループ化するよう指示します。これを行うため、SASは自動的にBY文の変数名ごとに2つの一時変数FIRST.変数とLAST.変数を作成します。これらの変数は0または1を取ります。

FIRST.変数 = 1 ならばその観測値はBYグループの最初の観測値です
FIRST.変数 = 0 ならばその観測値はBYグループの最初の観測値ではありません
LAST.変数 = 1 ならばその観測値はBYグループの最後の観測値です
LAST.変数 = 0 ならばその観測値はBYグループの最後の観測値ではありません

SASはFIRST.変数とLAST.変数の値を使って、BYグループの最初と最後の観測値、したがってグループ自体を識別します。一時的という形容詞についてひとこと付け加えると...SASはFIRST.変数とLAST.変数をプログラムデータベクターに配置するので、データステップでプログラミングが可能です。しかし、SASはこれらの変数を作成されるSASデータセットには追加しません。その意味で一時的なのです。

SASはFIRST.変数とLAST.変数を出力データセットに書き込まないので、これらの内容を見るにはいくつかの工夫が必要です。次の代入文

    firstGrade = FIRST.idno;
    lastGrade = LAST.idno;

はSASに対し、一時変数FIRST.idnoとLAST.idnoの値を永続的な変数firstGradeとlastGradeに割り当てるよう指示しています。PRINT手順は、SASにfirstGradeとlastGradeの値を含むデータセットを出力するよう指示しているので、FIRST.変数とLAST.変数の値を確認できます。


<!-- 
Because we are doing BY group processing on the variable idno, we must have the dataset sorted by idno. In this case the dataset was actually already sorted by idno, but I added the PROC SORT anyway to emphasize that the dataset **must be sorted first**.  
The SET and BY statement tell SAS to process the data by grouping observations with the same idno together. To do this, SAS automatically creats two temporary variables for each variable name in the BY statement. One of the temporary variables is called **FIRST.variable** , where _variable_ is the variable name appearing the BY statement. The other temporary variable is called **LAST.variable**. Both take the values 0 or 1:  

* FIRST.variable = 1 when an observation is the first observation in a BY group
* FIRST.variable = 0 when an observation is not the first observation in a BY group
* LAST.variable = 1 when an observation is the last observation in a BY group
* LAST.variable = 0 when an observation is not the last observation in a BY group

SAS uses the values of the FIRST.variable and LAST.variable temporary variables to identify the first and last observations in a group, and therefore the group itself. Oh, a comment about that adjective temporary ... SAS places FIRST.variable and LAST.variable in the program data vector and they are therefore available for DATA step programming, but SAS does not add them to the SAS data set being created. It is in that sense that they are temporary.  
Because SAS does not write FIRST.variables and LAST.variables to output data sets, we have to do some finagling to see their contents. The two assignment statements:  

``` 
    firstGrade = FIRST.idno; 
    lastGrade = LAST.idno;
```

simply tell SAS to assign the values of the temporary variables, FIRST.idno and LAST.idno, to permanent variables, firstGrade and lastGrade, respectively. The PRINT procedure tells SAS to print the resulting data set so that we can take an inside peek at the values of the FIRST.variables and LAST.variables.  
-->

例
RETAIN文の最も強力な使い方の1つは、観測値間で値を比較することです。次のプログラムは、RETAIN文を使って観測値間の値を比較し、各被験者の4回の試験のうち最低の成績を決定しています。

(コードは省略)


<!-- 
### Example

One of the most powerful uses of a RETAIN statement is to compare values across observations. The following program uses the RETAIN statement to compare values across observations, and in doing so determines each student's lowest grade of the four semester exams:  
-->

In [20]:
DATA exams;
    set grades (where = (gtype in ('E1', 'E2', 'E3', 'E4')));
RUN;
 
DATA lowest (rename = (lowtype = gtype));
    set exams;
    by idno;
    retain lowgrade lowtype;
    if first.idno then lowgrade = grade;
    lowgrade = min(lowgrade, grade);
    if grade = lowgrade then lowtype = gtype;
    if last.idno then output;
    drop gtype;
RUN;
 
PROC PRINT data=lowest;
    title 'Output Dataset: LOWEST';
RUN;

Obs,idno,l_name,grade,lowgrade,gtype
1,10,Smith,69,69,E4
2,11,Simon,99,72,E2
3,12,Jones,99,92,E3


最初のデータステップでは、gradesデータセットから試験の成績(E1、E2、E3、E4)のみを抽出してexamsデータセットを作成しています。

2番目のデータステップが本質的な部分で、理解が難しいかもしれません。このデータステップでは、examsデータセットを検索し、各被験者(「by idno」)について最低の成績(「min(lowgrade, grade)」)を探します。SASは通常、lowgradeとlowtypeの変数をデータステップの各反復の開始時にミッシング値にリセットするので、RETAIN文を使ってこれらの変数の最低値を保持しています。最後の観測値(「last.idno」)を読み込むと、最低の試験種類(lowtype)と成績(lowgrade)に対応するデータがlowestデータセットに出力されます。(「if last.idno then output;」という文は、観測値を被験者ごとに1つにまとめる効果があります。) lowestデータセットをgradesデータセットにidnoとgtypeで結合できるよう、lowtypeの変数名をgtypeに変更しています。
<!-- 
Because the instructor only wants to drop the lowest exam grade, the first DATA step tells SAS to create a data set called exams by selecting only the exam grades (E1, E2, E3, and E4) from the data set grades.  
It's the second DATA step that is the meat of the program and the challenging one to understand. The DATA step searches through the exams data set for each subject ("by idno") and looks for the lowest grade ("min(lowgrade, grade)"). Because SAS would otherwise set the variables lowgrade and lowtype to missing for each new iteration, the RETAIN statement is used to keep track of the observation that contains the lowest grade. When SAS reads the last observation of the student ("last.idno") it outputs the data corresponding to the lowest exam type (lowtype) and grade (lowgrade) to the lowest data set. (Note that the statement "if last.idno then output;" effectively collapses multiple observations per student into one observation per student.) So that we can merge the lowest data set back into the grades data set, by idno and gtype, the variable lowtype is renamed back to gtype.  
-->

## DOループ 

プログラミングをする際、同じ文を繰り返し実行する必要がある場合があります。そんな時にDOループが活躍します。DOループの中には、無条件で実行されるものがあります。20回実行するよう指示すれば、SASは20回実行します。このようなループを**反復型DOループ**と呼びます。一方で、特定の条件が満たされるまで実行を続けるDOループや、特定の条件が満たされている間実行を続けるDOループもあります。前者をDO UNTILループ、後者をDO WHILEループと呼びます。このレッスンでは、これら3種類のループの概要と多くの例を見ていきます。次のセクションでは、DOループを使って配列を処理する方法を学びます。

### 反復型DOループ

このセクションでは、反復型DOループ、つまりSASに対して文やグループの文を特定の回数実行するよう指示する方法を見ていきます。いくつかの例を見てみましょう。

例
次のプログラムは、DOループを使って4×3がいくつになるかをSASに計算させています。  

<!-- 
## DO Loops

When programming, you can find yourself needing to tell SAS to execute the same statements over and over again. That's when a DO loop can come in and save your day. The actions of some DO loops are unconditional in that if you tell SAS to do something 20 times, SAS will do it 20 times regardless. We call those kinds of loops **iterative DO loops**. On the other hand, actions of some DO loops are conditional in that you tell SAS to do something until a particular condition is met or to do something while a particular condition is met. We call the former a **DO UNTIL** loop and the latter a **DO WHILE** loop. In this lesson, we'll explore the ins and outs of these three different kinds of loops, as well as take a look at lots of examples in which they are used. Then, in the next section, we'll use DO loops to help us process arrays.

### Iterative DO Loops

In this section, we'll explore the use of iterative DO loops, in which you tell SAS to execute a statement or a group of statements a certain number of times. Let's take a look at some examples.

### Example

The following program uses a DO loop to tell SAS to determine what four times three (4 × 3) equals:  
-->

In [21]:
DATA multiply;
    answer = 0;
    do i = 1 to 4;
        answer + 3;
    end;
RUN;
 
PROC PRINT NOOBS;
    title 'Four Times Three Equals...';
RUN;

answer,i
12,5


確かに、4×3を計算するにはもっと簡単な方法がありますが、それではDOループを使う楽しみがありません!このデータステップを理解するカギは、掛け算は足し算の繰り返しだということを思い出すことです。つまり、4×3は3+3+3+3と同じことです。データステップのDOループは、SASにこの計算を行うよう指示しているだけなのです。最初にanswerに0を代入した後、3をanswerに加え、さらに3を加え、さらに3を加え、さらに3を加えます。SASがanswerに3を4回加えた後、DOループから抜け出し、データステップの終わりなのでSASは次の手順に進みます。 


データステップについて気づくべきもう一つのことは、入力データセットや入力データファイルがないことです。ここでは、何らかの入力ソースからではなく、データをゼロから生成しています。SASプログラムを起動して実行し、PRINTプロシージャの出力を確認して、コードが正しく四掛ける三を計算していることを確認してください。

さて、私たちのmultiplyデータセットに表示されるその変数iについてはどうでしょうか？もう一度データステップを見ると、それがDOループから来ていることがわかります。これはインデックス変数（またはカウンター変数）と呼ばれます。通常、出力データセットから削除することをお勧めしますが、ここでの存在は教育的です。ご覧の通り、その現在の値は5です。これがSASがDOループを終了するのを可能にしています... SASにはiが4になるまでループ内のアクションを取るように指示しています。iが4を超えると、SASはループを抜け、データステップの次の文に進みます。反復DOループの一般的な形式を見てみましょう。

<!-- 
Okay... admittedly, we could accomplish our goal of determining four times three in a much simpler way, but then we wouldn't have the pleasure of seeing how we can accomplish it using an iterative DO loop! The key to understanding the DATA step here is to recall that multiplication is just repeated addition. That is, four times three (4 × 3) is the same as adding three together four times, that is, 3 + 3 + 3 + 3. That's all that the iterative DO loop in the DATA step is telling SAS to do. After having initialized answer to 0, add 3 to answer, then add 3 to answer again, and add 3 to answer again, and add 3 to answer again. After SAS has added 3 to the answer variable four times, SAS exits the DO loop, and since that's the end of the DATA step, SAS moves onto the next procedure and prints the result.  
The other thing you might want to notice about the DATA step is that there is no input data set or input data file. We are generating data from scratch here, rather than from some input source. Now, launch and run the SAS program, and review the output from the PRINT procedure to convince yourself that our code properly calculates four times three.  
Ahhh, what about that _i_ variable that shows up in our multiply data set? If you look at our DATA step again, you can see that it comes from the DO loop. It is what is called the **index variable** (or **counter variable** ). Most often, you'll want to drop it from your output data set, but its presence here is educational. As you can see, its current value is 5. That's what allows SAS to exit the DO loop... we tell SAS only to take the actions inside the loop until _i_ equals 4. Once _i_ becomes greater than 4, SAS jumps out of the loop, and moves on to the next statements in the DATA step. Let's take a look at the general form of iterative DO loops.  
-->

反復DOループを構築するには、DO文で開始し、いくつかのアクション文を含め、END文で終了する必要があります。以下は単純な反復DOループの例です：

```sas
DO index-variable = start TO stop BY increment;
    action statements;
END;
```

ここで、

* DO、index-variable、start、TO、stop、およびENDはすべての反復DOループで必須です
* index-variableはDOループの現在の反復の値を格納する任意の有効なSAS変数名であり、通常は一文字、特にiとjがよく使われます
* startはループを開始するインデックス変数の値です
* stopはループを終了するインデックス変数の値です
* incrementは各反復後にSASがインデックス変数を変更する量です。最も一般的に使用される増分は1です。実際、BY句を指定しない場合、SASはデフォルトの増分1を使用します。

例えば、

```sas
do jack = 1 to 5;
```

はSASにジャックというインデックス変数を作成し、1から始まり、1ずつ増加し、5で終了するように指示します。したがって、ジャックの値は反復ごとに1、2、3、4、および5です。このDO文：

```sas
do jill = 2 to 12 by 2;
```

はSASにジルというインデックス変数を作成し、2から始まり、2ずつ増加し、12で終了するように指示します。したがって、ジルの値は反復ごとに2、4、6、8、10、および12です。

### 例
次のSASプログラムは反復DOループを使用して1ずつカウントダウンします：



<!-- 
To construct an iterative DO loop, you need to start with a DO statement, then include some action statements, and then end with an END statement. Here's what a simple iterative DO loop should look like:

```
DO index-variable = start TO stop BY increment;
    action statements;
END;
``` 

where

* DO, index-variable, start, TO, stop, and END are required in every iterative DO loop
* index-variable, which stores the value of the current iteration of the DO loop, can be any valid SAS variable name. It is common, however, to use a single letter, with i and j being the most used.
* start is the value of the index variable at which you want SAS to start the loop
* stop is the value of the index variable at which you want SAS to stop the loop
* increment is by how much you want SAS to change the index variable after each iteration. The most commonly used increment is 1. In fact, if you don't specify a BY clause, SAS uses the default increment of 1.

For example,

`do jack = 1 to 5;`

tells SAS to create an index variable called jack, start at 1, increment by 1, and end at 5, so that the values of jack from iteration to iteration are 1, 2, 3, 4, and 5. And, this DO statement:

`do jill = 2 to 12 by 2;`

tells SAS to create an index variable called jill, start at 2, increment by 2, and end at 12, so that the values of jill from iteration to iteration are 2, 4, 6, 8, 10, and 12.

### Example

The following SAS program uses an iterative DO loop to count backwards by 1:  
-->

In [22]:
DATA backwardsbyone;
    do i = 20 to 1 by -1;
        output;
    end;
RUN;
 
PROC PRINT data = backwardsbyone NOOBS;
    title 'Counting Backwards by 1';
RUN;

i
20
19
18
17
16
15
14
13
12
11



ご覧のように、このDO文では、BY句に負の値を指定することでDOループのインデックス変数を減少させることができます。ここでは、SASに20から始まり、インデックス変数を1ずつ減少させ、1に達するまでループするように指示しています。OUTPUT文は、DOループの各反復でインデックス変数iの値を出力するようにSASに指示します。SASプログラムを起動して実行し、PRINTプロシージャの出力を確認して、コードが20から1まで正しくカウントダウンしていることを確認してください。


<!-- 
As you can see in this DO statement, you can decrement a DO loop's index variable by specifying a negative value for the BY clause. Here, we tell SAS to start at 20, and decrease the index variable by 1, until it reaches 1. The OUTPUT statement tells SAS to output the value of the index variable i for each iteration of the DO loop. Launch and run the SAS program, and review the output from the PRINT procedure to convince yourself that our code properly counts backwards from 20 to 1.  
-->

DO文で開始、停止、および増分値を指定する代わりに、シリーズ内の項目をリストしてDOループを何回実行するかをSASに指示することができます。この場合、反復DOループの一般的な形式は次のようになります：

```sas
DO index-variable = value1, value2, value3, ...;
    action statements;
END;
```

ここで、値は文字または数値のいずれかです。DOループが実行されると、シリーズ内の各項目に対して一度ずつ実行されます。インデックス変数は現在の項目の値に等しくなります。シリーズ内の項目をリストするには、カンマで区切る必要があります。シリーズ内の項目をリストするには、すべての数値を指定する必要があります：

```sas
DO i = 1, 2, 3, 4, 5;
```

すべての文字値は、各値を引用符で囲みます：

```sas
DO j = 'winter', 'spring', 'summer', 'fall';
```

またはすべての変数名：

```sas
DO k = first, second, third;
```

この場合、インデックス変数は指定された変数の値を取ります。変数名は引用符で囲まれていませんが、文字値には引用符が必要です。

### ネストされたDOループ

他のプログラミング言語と同様に、ループを互いにネストすることができます。

### 例
たとえば、AとBという2つの要因を使って実験を行いたいとします。要因Aは、例えば、水の量であり、レベルは1、2、3、および4です。要因Bは、例えば、日光の量であり、レベルは1、2、3、4、および5です。次のSASコードはネストされた反復DOループを使用して4x5の因子設計を生成します：


<!-- 
Rather than specifying start, stop and increment values in a DO statement, you can tell SAS how many times to execute a DO loop by listing items in a series. In this case, the general form of the iterative DO loop looks like this:

```
DO index-variable = value1, value2, value3, ...;
    action statements;
END;
```

where the values can be character or numeric. When the DO loop executes, it executes once for each item in the series. The index variable equals the value of the current item. You must use commas to separate items in a series. To list items in a series, you must specify

either all numeric values: 

`DO i = 1, 2, 3, 4, 5;`

all character values, with each value enclosed in quotation marks 

`DO j = 'winter', 'spring', 'summer', 'fall';`

or all variable names: 

`DO k = first, second, third;`

In this case, the index variable takes on the values of the specified variables. Note that the variable names are not enclosed in quotation marks, while quotation marks are required for character values.

### Nested DO Loops

Just like in other programming languages. We can nest loops within each other.

### Example

Suppose you are interested in conducting an experiment with two factors A and B. Suppose factor A is, say, the amount of water with levels 1, 2, 3, and 4; and factor B is, say, the amount of sunlight, say with levels 1, 2, 3, 4, and 5. Then, the following SAS code uses nested iterative DO loops to generate the 4 by 5 factorial design:  
-->

In [23]:
DATA design;
DO i = 1 to 4;
    DO j = 1 to 5;
        output;
            END;
    END;
RUN;
 
PROC PRINT data = design;
    TITLE '4 by 5 Factorial Design';
RUN;

Obs,i,j
1,1,1
2,1,2
3,1,3
4,1,4
5,1,5
6,2,1
7,2,2
8,2,3
9,2,4
10,2,5



まず、SASプログラムを起動して実行します。その後、PRINTプロシージャの出力を確認して、デザインデータセットの内容を確認してください。これにより、ネストされたDOループの動作についての理解が深まります。まず、SASはインデックス変数iの値を1に設定し、次にもう一つの反復DOループに進みます。iが1の間、SASはjの値を1に設定し、i=1およびj=1の観測値を出力します。その後、SASはjの値を2に設定し、i=1およびj=2の観測値を出力します。このプロセスは、jが5に設定され、iの値が2に設定されるまで続き、同様の方法で再び繰り返されます。SASはインデックス変数iの値が4になるまで、このプロセスを繰り返し、外側のDOループを抜け、データステップを終了します。


<!-- 
First, launch and run the SAS program. Then, review the output from the PRINT procedure to see the contents of the design data set. By doing so, you can get a good feel for how the nested DO loops work. First, SAS sets the value of the index variable i to 1, then proceeds to the next step which happens to be another iterative DO loop. While i is 1:  

* SAS sets the value of j to 1, and outputs the observation in which i = 1 and j = 1.
* Then, SAS sets the value j to 2, and outputs the observation in which i = 1 and j = 2.
* Then, SAS sets the value j to 3, and outputs the observation in which i = 1 and j = 3.
* Then, SAS sets the value j to 4, and outputs the observation in which i = 1 and j = 4.
* Then, SAS sets the value j to 5, and outputs the observation in which i = 1 and j = 5.
* Then, SAS sets the value j to 6, and jumps out of the inside DO loop and proceeds to the next statement, which happens to be the end of the outside DO loop.

SAS then sets the value of the index variable i to 2, then proceeds through the inside DO loop again just as described above. This process continues until SAS sets the value of index variable i to 5, jumps out of the outside DO loop, and ends the DATA step.  
-->

### DO UNTILおよびDO WHILEループ

反復DOループでは、DOループの反復回数を指定する必要があることがわかっています。しかし、条件が達成されるまで、または条件が存在する間、DOループを実行したいが、必要な反復回数がわからない場合があります。その場合、DO UNTILループおよびDO WHILEループが役立ちます。

このセクションでは、まずDO UNTILおよびDO WHILEループについて学びます。次に、条件付きおよび無条件のDOループの両方の機能を組み合わせた反復DOループの別の形式を見てみましょう。

DO UNTILループを使用する場合、SASは指定した式が真になるまでDOループを実行します。DO UNTILループの一般的な形式は次のようになります：

```sas
DO UNTIL (expression);
    action statements;
END;
```

ここで、expressionは括弧で囲まれた任意の有効なSAS式です。重要なことは、式はループの最後まで評価されないことです。したがって、DO UNTILループは必ず少なくとも一度実行されます。式が真と判断されたら、DOループは再び実行されません。

### 例
毎年1200を口座に預け、5％の利子が付く場合

、5年以内に口座残高が6000を超えるかどうかを知りたいとします。以下のSASコードはDO UNTILループを使用してこのシナリオをシミュレートします：


<!-- 
### DO UNITL and DO WHILE Loops

As you now know, the iterative DO loop requires that you specify the number of iterations for the DO loop. However, there are times when you want to execute a DO loop until a condition is reached or while a condition exists, but you don't know how many iterations are needed. That's when the DO UNTIL loop and the DO WHILE loop can help save the day!

In this section, we'll first learn about the DO UNTIL and DO WHILE loops. Then, we'll look at another form of the iterative DO loop that combines features of both conditional and unconditional DO loops.

When you use a DO UNTIL loop, SAS executes the DO loop until the expression you've specified is true. Here's the general form of a DO UNTIL loop:

```    
DO UNTIL (expression);
    action statements;
END;
```

where expression is any valid SAS expression enclosed in parentheses. The key thing to remember is that the expression is not evaluated until the bottom of the loop. Therefore, a DO UNTIL loop always executes at least once. As soon as the expression is determined to be true, the DO loop does not execute again.

### Example

Suppose you want to know how many years it would take to accumulate 50,000 if you deposit 1200 each year into an account that earns 5% interest. The following program uses a DO UNTIL loop to perform the calculation for us:  
-->

In [24]:
DATA investment;
    RETAIN value 0 year 0;
    DO UNTIL (value >= 50000);
        value = value + 1200;
        value = value + value * 0.05;
        year = year + 1;
        OUTPUT;
    END;
RUN;
 
PROC PRINT data = investment NOOBS;
    title 'Years until at least $50,000';
RUN;

value,year
1260.00,1
2583.00,2
3972.15,3
5430.76,4
6962.30,5
8570.41,6
10258.93,7
12031.88,8
13893.47,9
15848.14,10


Recall that the expression in the DO UNTIL statement is not evaluated until the bottom of the loop. Therefore, the DO UNTIL loop executes at least once. On the first iteration, the value variable is increased by 1200, or in this case, set to 1200. Then, the value variable is updated by calculating 1200 + 1200*0.05 to get 1260. Then, the year variable is increased by 1, or in this case, set to 1. The first observation, for which year = 1 and value = 1260, is then written to the output data set called investment. Having reached the bottom of the DO UNTIL loop, the expression (value >= 50000) is evaluated to determine if it is true. Since value is just 1260, the expression is not true, and so the DO UNTIL loop is executed once again. The process continues as described until SAS determines that value is at least 50000 and therefore stops executing the DO UNTIL loop.  
Launch and run the SAS program, and review the output from the PRINT procedure to convince yourself that it would take 23 years to accumulate at least $50,000.  

まず、SASプログラムを起動して実行します。その後、PRINTプロシージャの出力を確認して、5年以内に口座残高が6000を超えるかどうかを確認してください。ご覧の通り、DO文にはTO句とUNTIL句の両方が含まれています。DOループは年が1から5まで実行され、balanceが6000を超える場合はループが終了します。


ここで、expressionは括弧で囲まれた任意の有効なSAS式です。DO WHILEループとは異なり、DO UNTILループは式が最初に評価される前に少なくとも一度実行されます。

### 例
毎年1200を口座に預け、5％の利子が付く場合、口座残高が6000を超えるまでの年数を知りたいとします。以下のSASコードはDO WHILEループを使用してこのシナリオをシミュレートします：



まず、SASプログラムを起動して実行します。その後、PRINTプロシージャの出力を確認して、口座残高が6000を超えるまでの年数を確認してください。

ここで、year変数を0に初期化する必要があります。そうでなければ、SASはyearの最初の値が欠損値であると解釈し、予期しない結果が得られる可能性があります。データステップを繰り返し、DO WHILEおよびDO UNTILループの違いを理解してください。



When you use a DO WHILE loop, SAS executes the DO loop while the expression you've specified is true. Here's the general form of a DO WHILE loop: 

```    
DO WHILE (expression);
      action statements;
END;
```

where expression is any valid SAS expression enclosed in parentheses. An important difference between the DO UNTIL and DO WHILE statements is that the DO WHILE expression is evaluated at the top of the DO loop. If the expression is false the first time it is evaluated, then the DO loop doesn't even execute once.

### Example

The following program attempts to use a DO WHILE loop to accomplish the same goal as the program above, namely to determine how many years it would take to accumulate \$50,000 if you deposit \$1200 each year into an account that earns 5% interest:  

In [27]:
DATA investtwo;
    RETAIN value 0 year 0;
    DO WHILE (value < 50000);
        value = value + 1200;
        value = value + value * 0.05;
        year = year + 1;
        OUTPUT;
     END;
RUN;
 
PROC PRINT data = investtwo NOOBS;
   title 'Years until at least $50,000';
RUN;

value,year
1260.00,1
2583.00,2
3972.15,3
5430.76,4
6962.30,5
8570.41,6
10258.93,7
12031.88,8
13893.47,9
15848.14,10




計算は前と同様に進行します。最初に、値の変数は0 + 1200の計算によって1200に更新されます。次に、値の変数は1200 + 1200*0.05の計算によって1260に更新されます。次に、year変数が1つ増えて、この場合は1に設定されます。年=1、値=1260の最初の観測値が、investthreeという出力データセットに書き込まれます。その後、SASは DO WHILE ループの先頭に戻り、(値 < 50000)の式が真であるかどうかを判断します。値はまだ1260なので、式は真であり、したがってDO WHILEループがもう一度実行されます。この過程は、SASが値が少なくとも50000であると判断するまで続きます。その時点でDO WHILEループの実行を止めます。

SASプログラムを起動して実行し、PRINT手順の出力を確認して、このプログラムも少なくとも$50,000を蓄積するのに23年かかることを確認してください。

WHILE条件を値 < 50000 から値 >= 50000 に変更して、何が起こるか試してみてください。(ヒント: 出力がないはずです。なぜでしょうか?)



The calculations proceed as before. First, the value variable is updated to by calculating 0 + 1200, to get 1200. Then, the value variable is updated by calculating 1200 + 1200*0.05 to get 1260. Then, the year variable is increased by 1, or in this case, set to 1. The first observation, for which year = 1 and value = 1260, is then written to the output data set called investthree. SAS then returns to the top of the DO WHILE loop, to determine if the expression (value < 50000) is true. Since value is just 1260, the expression is true, and so the DO WHILE loop executes once again. The process continues as described until SAS determines that value is as least 50000 and therefore stops executing the DO WHILE loop.  
Launch and run the SAS program, and review the output from the PRINT procedure to convince yourself that this program also determines that it would take 23 years to accumulate at least \$50,000.  
You should also try changing the WHILE condition from value < 50000 to value ≥ 50000 to see what happens. (Hint: you will get no output. Why?)  

これまでに、DO WHILEループとDO UNTILループを使って、条件付きで文を繰り返し実行する方法を見てきました。また、反復DOループを使って、無条件で一定回数の文を実行する方法も見てきました。次に、条件付きでも無条件でも実行できるDOループの形式を作成します。

## 例

再び、1年目に1200ドルを預け入れ、5%の利息が付く口座に50,000ドルを蓄積するのにどれくらいの年数がかかるかを知りたいとします。しかし今度は、投資期間を15年に制限したいと想定します。次のプログラムでは、条件付き反復DOループを使って、15年に達するか、投資額が50,000ドルを超えるまで、投資額を蓄積します。


You have now seen how the DO WHILE and DO UNTIL loops enable you to execute statements repeatedly, but conditionally so. You have also seen how the iterative DO loop enables you to execute statements a set number of times unconditionally. Now, we'll put the two together to create a form of the iterative DO loop that executes DO loops conditionally as well as unconditionally.

### Example

Suppose again that you want to know how many years it would take to accumulate 50,000 if you deposit 1200 each year into an account that earns 5% interest. But this time, suppose you also want to limit the number of years that you invest to 15 years. The following program uses a conditional iterative DO loop to accumulate our investment until we reach 15 years or until the value of our investment exceeds 50000, whichever comes first:  

In [28]:
DATA investfour (drop = i);
     RETAIN value 0 year 0;
     DO i = 1 to 15 UNTIL (value >= 50000);
          value = value + 1200;
          value = value + value * 0.05;
          year = year + 1;
          OUTPUT;
     END;
RUN;
 
PROC PRINT data = investfour NOOBS;
   title 'Value of Investment';
RUN;

value,year
1260.00,1
2583.00,2
3972.15,3
5430.76,4
6962.30,5
8570.41,6
10258.93,7
12031.88,8
13893.47,9
15848.14,10


プログラムの注目点は2つあります。
i) 反復i = 1 to 15がDO UNTILステートメントに挿入されています。
ii)インデックス変数iがDOループ用に作成されたため、プログラムデータベクトルの内容をinvestfour出力データセットに書き込む前に削除されています。

Note that there are just two differences between this program and that of the program in the previous example that uses the DO UNTIL loop: i) The iteration i = 1 to 15 has been inserted into the DO UNTIL statement; and ii) because the index variable i is created for the DO loop, it is dropped before writing the contents from the program data vector to the output data set investfour.  


## SAS配列

この節では、SASでの基本的な配列処理について学びます。DATA ステッププログラミングでは、複数の変数に同じ処理を一度に行う必要がしばしばあります。変数を個別に処理することも可能ですが、通常はグループとして扱う方が簡単です。配列はそのためのオプションを提供します。例えば、今までSASデータセットの50の数値変数の平方根を求めたい場合、50のSAS代入文を書く必要がありました。代わりに配列を使用すると、この作業を簡単にできます。

配列は、次のようなタスクを簡素化したい場合に使用できます。

* 繰り返し計算を実行する
* 同じ属性を持つ多くの変数を作成する
* データを読み込む
* 「太った」データセットを「細長い」データセットに転置する(データセットの変数を観測値に変換する)
* 「細長い」データセットを「太った」データセットに転置する(データセットの観測値を変数に変換する)
* 変数を比較する

このレッスンでは、そのようなタスクをどのように実行するかを学びます。適切な場面で配列を使用すると、SASプログラムを大幅に簡略化・短縮化できます!

### 1次元配列

SAS **配列** は、単一の名前の下に一時的にSAS変数をグループ化したものです。例えば、winter、spring、summer、fallという4つの変数がある場合、seasonsという配列名でそれらの変数を関連付け、seasons(1)、seasons(2)、seasons(3)、seasons(4) として参照できます。配列を反復DOループと組み合わせると、コンピュータプログラムを効率的に記述できる強力なツールになります。例を見てみましょう。

## 例

次のプログラムは単に、米国の10都市の平均月別気温(セ氏)をavgcelsiusという一時的なSASデータセットに読み込むものです。

## SAS Arrays

In this section, we'll learn about basic array processing in SAS. In DATA step programming, you often need to perform the same action on more than one variable at a time. Although you can process the variables individually, it is typically easier to handle the variables as a group. Arrays offer you that option. For example, until now, if you wanted to take the square root of the 50 numeric variables in your SAS data set, you'd have to write 50 SAS assignment statements to accomplish the task. Instead, you can use an array to simplify your task.

Arrays can be used to simplify your code when you need to:

* perform repetitive calculations
* create many variables that have the same attributes
* read data
* transpose "fat" data sets to "tall" data sets, that is, change the variables in a data set to observations
* transpose "tall" data sets to "fat" data sets, that is, change the observations in a data set to variables
* compare variables

In this lesson, we'll learn how to accomplish such tasks using arrays. Using arrays in appropriate situations can seriously simplify and shorten your SAS programs!

### One-Dimensional Arrays

A SAS **array** is a temporary grouping of SAS variables under a single name. For example, suppose you have four variables named winter, spring, summer, and, fall. Rather than referring to the variables by their four different names, you could associate the variables with an array name, say seasons, and refer to the variables as seasons(1), seasons(2), seasons(3), and seasons(4). When you pair an array up with an iterative DO loop, you create a powerful and efficient way of writing your computer programs. Let's take a look at an example!

### Example

The following program simply reads in the average montly temperatures (in Celsius) for ten different cities in the United States into a temporary SAS data set called avgcelsius:  

In [29]:
DATA avgcelsius;
    input City $ 1-18 jan feb mar apr may jun
                        jul aug sep oct nov dec;
    DATALINES;
State College, PA  -2 -2  2  8 14 19 21 20 16 10  4 -1
Miami, FL          20 20 22 23 26 27 28 28 27 26 23 20
St. Louis, MO      -1  1  6 13 18 23 26 25 21 15  7  1
New Orleans, LA    11 13 16 20 23 27 27 27 26 21 16 12
Madison, WI        -8 -5  0  7 14 19 22 20 16 10  2 -5
Houston, TX        10 12 16 20 23 27 28 28 26 21 16 12
Phoenix, AZ        12 14 16 21 26 31 33 32 30 23 16 12
Seattle, WA         5  6  7 10 13 16 18 18 16 12  8  6
San Francisco, CA  10 12 12 13 14 15 15 16 17 16 14 11
San Diego, CA      13 14 15 16 17 19 21 22 21 19 16 14
;
RUN;
 
PROC PRINT data = avgcelsius;
    title 'Average Monthly Temperatures in Celsius';
    id City;
    var jan feb mar apr may jun 
        jul aug sep oct nov dec;
RUN;

City,jan,feb,mar,apr,may,jun,jul,aug,sep,oct,nov,dec
"State College, PA",-2,-2,2,8,14,19,21,20,16,10,4,-1
"Miami, FL",20,20,22,23,26,27,28,28,27,26,23,20
"St. Louis, MO",-1,1,6,13,18,23,26,25,21,15,7,1
"New Orleans, LA",11,13,16,20,23,27,27,27,26,21,16,12
"Madison, WI",-8,-5,0,7,14,19,22,20,16,10,2,-5
"Houston, TX",10,12,16,20,23,27,28,28,26,21,16,12
"Phoenix, AZ",12,14,16,21,26,31,33,32,30,23,16,12
"Seattle, WA",5,6,7,10,13,16,18,18,16,12,8,6
"San Francisco, CA",10,12,12,13,14,15,15,16,17,16,14,11
"San Diego, CA",13,14,15,16,17,19,21,22,21,19,16,14


私たちは摂氏温度をあまり理解できないため、華氏温度に変換したい場合があります。華氏温度の方が分かりやすいからです。次のSASプログラムでは、標準的な変換式:

華氏温度 = 1.8*摂氏温度 + 32

を使って、avgcelsiusデータセットの摂氏温度を華氏温度に変換し、新しいデータセットavgfahrenheitに格納します。


Now, suppose that we don't feel particularly comfortable with understanding Celsius temperatures, and therefore, we want to convert the Celsius temperatures into Fahrenheit temperatures for which we have a better feel. The following SAS program uses the standard conversion formula:  

``` 
Fahrenheit temperature = 1.8*Celsius temperature + 32
```

to convert the Celsius temperatures in the avgcelsius data set to Fahrenheit temperatures stored in a new data set called avgfahrenheit:  

In [30]:
DATA avgfahrenheit;
    set avgcelsius;
    janf = 1.8*jan + 32;
    febf = 1.8*feb + 32;
    marf = 1.8*mar + 32;
    aprf = 1.8*apr + 32;
    mayf = 1.8*may + 32;
    junf = 1.8*jun + 32;
    julf = 1.8*jul + 32;
    augf = 1.8*aug + 32;
    sepf = 1.8*sep + 32;
    octf = 1.8*oct + 32;
    novf = 1.8*nov + 32;
    decf = 1.8*dec + 32;
    drop jan feb mar apr may jun
            jul aug sep oct nov dec;
RUN;
 
PROC PRINT data = avgfahrenheit;
    title 'Average Monthly Temperatures in Fahrenheit';
    id City;
    var janf febf marf aprf mayf junf 
        julf augf sepf octf novf decf;
RUN;

City,janf,febf,marf,aprf,mayf,junf,julf,augf,sepf,octf,novf,decf
"State College, PA",28.4,28.4,35.6,46.4,57.2,66.2,69.8,68.0,60.8,50.0,39.2,30.2
"Miami, FL",68.0,68.0,71.6,73.4,78.8,80.6,82.4,82.4,80.6,78.8,73.4,68.0
"St. Louis, MO",30.2,33.8,42.8,55.4,64.4,73.4,78.8,77.0,69.8,59.0,44.6,33.8
"New Orleans, LA",51.8,55.4,60.8,68.0,73.4,80.6,80.6,80.6,78.8,69.8,60.8,53.6
"Madison, WI",17.6,23.0,32.0,44.6,57.2,66.2,71.6,68.0,60.8,50.0,35.6,23.0
"Houston, TX",50.0,53.6,60.8,68.0,73.4,80.6,82.4,82.4,78.8,69.8,60.8,53.6
"Phoenix, AZ",53.6,57.2,60.8,69.8,78.8,87.8,91.4,89.6,86.0,73.4,60.8,53.6
"Seattle, WA",41.0,42.8,44.6,50.0,55.4,60.8,64.4,64.4,60.8,53.6,46.4,42.8
"San Francisco, CA",50.0,53.6,53.6,55.4,57.2,59.0,59.0,60.8,62.6,60.8,57.2,51.8
"San Diego, CA",55.4,57.2,59.0,60.8,62.6,66.2,69.8,71.6,69.8,66.2,60.8,57.2


プログラムを見ると、変換に必要な代入文の数から、この作業は根気が必要であることが分かります。平均月別気温が12個あるため、12の代入文を書かなければなりません。各代入文は同じ計算を実行します。変数名だけが異なります。SASプログラムを起動して実行し、PRINT手順の出力を確認して、摂氏温度が適切に華氏温度に変換されたことを確認してください。

上記のプログラムは配列の使用を求めています。配列を使う主な理由の1つは、変数の処理に必要なステートメントの数を減らすことです。例を見てみましょう。

次のプログラムでは、1次元配列fahrを使って、avgcelsiusデータセットの平均摂氏温度を平均華氏温度に変換し、新しいデータセットavgfahrenheitに格納します。


As you can see by the number of assignment statements necessary to make the conversions, the exercise becomes one of patience. Because there are twelve average monthly temperatures, we must write twelve assignment statements. Each assignment statement performs the same calculation. Only the name of the variable changes in each statement. Launch and run the SAS program, and review the output from the PRINT procedure to convince yourself that the Celsius temperatures were properly converted to Fahrenheit temperatures.  
The above program is crying out for the use of an array. One of the primary arguments for using an array is to reduce the number of statements that are required for processing variables. Let's take a look at an example.  
The following program uses a one-dimensional array called fahr to convert the average Celsius temperatures in the avgcelsius data set to average Fahrenheit temperatures stored in a new data set called avgfahrenheit:  

In [31]:
DATA avgfahrenheit;
    set avgcelsius;
    array fahr(12) jan feb mar apr may jun
                   jul aug sep oct nov dec;
    do i = 1 to 12;
        fahr(i) = 1.8*fahr(i) + 32;
    end;
RUN;
 
PROC PRINT data = avgfahrenheit;
    title 'Average Monthly Temperatures in Fahrenheit';
    id City;
    var jan feb mar apr may jun 
        jul aug sep oct nov dec;
RUN;

City,jan,feb,mar,apr,may,jun,jul,aug,sep,oct,nov,dec
"State College, PA",28.4,28.4,35.6,46.4,57.2,66.2,69.8,68.0,60.8,50.0,39.2,30.2
"Miami, FL",68.0,68.0,71.6,73.4,78.8,80.6,82.4,82.4,80.6,78.8,73.4,68.0
"St. Louis, MO",30.2,33.8,42.8,55.4,64.4,73.4,78.8,77.0,69.8,59.0,44.6,33.8
"New Orleans, LA",51.8,55.4,60.8,68.0,73.4,80.6,80.6,80.6,78.8,69.8,60.8,53.6
"Madison, WI",17.6,23.0,32.0,44.6,57.2,66.2,71.6,68.0,60.8,50.0,35.6,23.0
"Houston, TX",50.0,53.6,60.8,68.0,73.4,80.6,82.4,82.4,78.8,69.8,60.8,53.6
"Phoenix, AZ",53.6,57.2,60.8,69.8,78.8,87.8,91.4,89.6,86.0,73.4,60.8,53.6
"Seattle, WA",41.0,42.8,44.6,50.0,55.4,60.8,64.4,64.4,60.8,53.6,46.4,42.8
"San Francisco, CA",50.0,53.6,53.6,55.4,57.2,59.0,59.0,60.8,62.6,60.8,57.2,51.8
"San Diego, CA",55.4,57.2,59.0,60.8,62.6,66.2,69.8,71.6,69.8,66.2,60.8,57.2


このプログラムを前のプログラムと比較すると、12の代入文を置き換えたステートメントが分かります。ARRAYステートメントは、fahrという配列を定義します。jan、feb、...、decの12か月の変数をfahrという配列にグループ化することをSASに伝えます。括弧()内の(12)は、配列宣言に必須の部分で、配列の次元と呼ばれます。これにより、SASにグループ化したい変数の数を知らせます。配列に含める変数名を指定する際は、スペースで区切って変数を列挙します。他のSASステートメントと同様に、ARRAYステートメントはセミコロン(;)で終わります。

一度fahrという配列を定義すれば、個々の変数名の代わりにコードで使用できます。配列の個々の要素は、fahr(i)のように名前とインデックスで参照します。ARRAYステートメントでの変数の並び順が、配列内の変数の位置を決めます。例えば、fahr(1)はjan変数、fahr(2)はfeb変数、fahr(12)はdec変数に対応します。配列をfahrのように反復DOループとともに使うと、コードを本当に簡略化できます。

DOループは、SASにfahrの配列の要素を処理させ、その都度摂氏温度を華氏温度に変換します。例えば、インデックス変数iが1の場合、代入文は次のようになります。

fahr(1) = 1.8*fahr(1) + 32;

つまり、

jan = 1.8*jan + 32;

と考えることができます。等号の右側のjanの値は摂氏温度です。代入文が実行された後、等号の左側のjanの値が更新され、華氏温度を反映します。

SASプログラムを起動して実行し、PRINT手順の出力を確認して、摂氏温度が再び適切に華氏温度に変換されたことを確認してください。最後に付け加えますが、PRINT手順のVARステートメントで列挙されている変数は、jan、feb、...、decの元の変数名であり、fahr(1)、fahr(2)、...のように配列でグループ化された変数ではありません。配列はDATAステップの間だけ存在するためです。PRINT手順でfahr(1)、fahr(2)、...と指定すると、SASはひっかかります。要約しましょう!



If you compare this program with the previous program, you can see the statements that replaced the twelve assignment statements. The ARRAY statement defines an array called fahr. It tells SAS that you want to group the twelve month variables, jan , feb, ... dec, into an array called fahr. The (12) that appears in parentheses is a required part of the array declaration. Called the dimension of the array, it tells SAS how many elements, that is, variables, you want to group together. When specifying the variable names to be grouped in the array, we simply list the elements, separating each element with a space. As with all SAS statements, the ARRAY statement is closed with a semicolon (;).  
Once we've defined the array fahr, we can use it in our code instead of the individual variable names. We refer to the individual elements of the array using its name and an index, such as, fahr(i). The order in which the variables appear in the ARRAY statement determines the variable's position in the array. For example, fahr(1) corresponds to the jan variable, fahr(2) corresponds to the feb variable, and fahr(12) corresponds to the dec variable. It's when you use an array like fahr, in conjunction with an iterative DO loop, that you can really simplify your code, as we did in this program.  
The DO loop tells SAS to process through the elements of the fahr array, each time converting the Celsius temperature to a Fahrenheit temperature. For example, when the index variable i is 1, the assignment statement becomes:  

```
fahr(1) = 1.8*fahr(1) + 32;
```

which you could think of as saying:  

``` 
jan = 1.8*jan + 32;
```

The value of jan on the right side of the equal sign is the Celsius temperature. After the assignment statement is executed, the value of jan on the left side of the equal sign is updated to reflect the Fahrenheit temperature.  
Now, launch and run the SAS program, and review the output from the PRINT procedure to convince yourself that the Celsius temperatures were again properly converted to Fahrenheit temperatures. Oh, one more thing to point out! Note that the variables listed in the PRINT procedure's VAR statement are the original variable names jan, feb, ..., dec, not the variables as they were grouped into an array, fahr(1), fahr(2), ..., fahr(12). That's because **an array exists only for the duration of the DATA step**. If in the PRINT procedure, you instead tell SAS to print fahr(1), fahr(2), ... you'll see that SAS will hiccup. Let's summarize!  

配列を定義するには、ARRAYステートメントを使って、以前に定義されたデータセット変数をグループ化する必要があります。ARRAYステートメントの一般的な形式は次の通りです。

`ARRAY 配列名(次元) ;`

ここで:

- 配列名は、配列の名前を指定する有効なSAS名でなければなりません。
- 次元は、配列要素の数と配列の構成を記述します。デフォルトの次元は1です。
- 要素は、配列を形成するためにグループ化される変数のリストです。配列要素はすべて数値またはすべて文字でなければなりません。標準的なSASヘルプ表記では、<>の角括弧で囲まれた用語は省略可能であることを示しています。つまり、ARRAYステートメントで要素を指定する必要はありません。要素が指定されていない場合、デフォルトの名前で新しい変数が作成されます。

配列名についてもう少し説明しておく必要があります。SASを混乱させたくない場合は、同じDATAステップ内に現れる変数と同じ名前の配列を作ってはいけません。また、有効なSAS関数と同じ名前の配列を作るのも避けるべきです。SASはそうすることを許可しますが、その場合、同じDATAステップ内でその関数を使用できなくなります。例えば、DATAステップでmeanという名前の配列を作った場合、そのDATAステップ内でmean関数を使うことはできません。SASはログウィンドウに警告メッセージを出力して知らせます。最後に、配列名はLABEL、FORMAT、DROP、KEEP、LENGTHステートメントでは使用できません。

別の方法で摂氏温度から華氏温度への変換に使用する配列を定義した例を見てみましょう。

## 例

次のプログラムは前の例と同じですが、ARRAYステートメントの12が*(アスタリスク)に変更されており、SASリストを使って配列の変数をキャプチャしています。


To define an array, you must use an ARRAY statement having the following general form in order to group previously defined data set variables into an array:

`ARRAY array-name(dimension) <elements>;`

where:

* array-name must be a valid SAS name that specifies the name of the array
* dimension describes the number and arrangement of array elements. The default dimension is one.
* elements list the variables to be grouped together to form the array. The array elements must be either all numeric or all character. Using standard SAS Help notation, the term elements appears in <> brackets to indicate that they are optional. That is, you do not have to specify elements in the ARRAY statement. If no elements are listed, new variables are created with default names.

A few more points must be made about the array-name. Unless you are interested in confusing SAS, you should not give an array the same name as a variable that appears in the same DATA step. You should also avoid giving an array the same name as a valid SAS function. SAS allows you to do so, but then you won't be able to use the function in the same DATA step. For example, if you named an array mean in a DATA step, you would not be able to use the mean function in the DATA step. SAS will print a warning message in your log window to let you know such. Finally, array names cannot be used in LABEL, FORMAT, DROP, KEEP, or LENGTH statements.

Let's look at another example to see a different way to define the array used to convert degrees Celsius to Farenheit.

### Example

The following program is identical to the program in the previous example, except the 12 in the ARRAY statement has been changed to an asterisk (*) and we use a SAS list to grab the variables for the array:  

In [32]:
DATA avgfahrenheittwo;
    set avgcelsius;
    array fahr(*) jan -- dec;
    do i = 1 to 12;
            fahr(i) = 1.8*fahr(i) + 32;
    end;
RUN;
 
PROC PRINT data = avgfahrenheittwo;
    title 'Average Monthly Temperatures in Fahrenheit';
    id City;
    var jan feb mar apr may jun 
        jul aug sep oct nov dec;
RUN;

City,jan,feb,mar,apr,may,jun,jul,aug,sep,oct,nov,dec
"State College, PA",28.4,28.4,35.6,46.4,57.2,66.2,69.8,68.0,60.8,50.0,39.2,30.2
"Miami, FL",68.0,68.0,71.6,73.4,78.8,80.6,82.4,82.4,80.6,78.8,73.4,68.0
"St. Louis, MO",30.2,33.8,42.8,55.4,64.4,73.4,78.8,77.0,69.8,59.0,44.6,33.8
"New Orleans, LA",51.8,55.4,60.8,68.0,73.4,80.6,80.6,80.6,78.8,69.8,60.8,53.6
"Madison, WI",17.6,23.0,32.0,44.6,57.2,66.2,71.6,68.0,60.8,50.0,35.6,23.0
"Houston, TX",50.0,53.6,60.8,68.0,73.4,80.6,82.4,82.4,78.8,69.8,60.8,53.6
"Phoenix, AZ",53.6,57.2,60.8,69.8,78.8,87.8,91.4,89.6,86.0,73.4,60.8,53.6
"Seattle, WA",41.0,42.8,44.6,50.0,55.4,60.8,64.4,64.4,60.8,53.6,46.4,42.8
"San Francisco, CA",50.0,53.6,53.6,55.4,57.2,59.0,59.0,60.8,62.6,60.8,57.2,51.8
"San Diego, CA",55.4,57.2,59.0,60.8,62.6,66.2,69.8,71.6,69.8,66.2,60.8,57.2


簡単ですね!変数の数を数えたり、配列にグループ化する変数を個別に列挙する必要なく、SASにその作業をさせることができます。そのためには、次元を*(アスタリスク)で定義し、SASリストのショートカットを使用します。配列にグループ化する変数が多すぎて、個別にカウントして列挙するのが面倒な場合、この方法が便利でしょう。ちなみに、このレッスンを通して、配列の次元(またはインデックス変数)を括弧()で囲んでいますが、中括弧{}または山括弧[]を使ってもかまいません。

上記のプログラムでは、SASリストを使って、fahr配列にグループ化された変数名のリストを短縮しました。場合によっては、_ALL_、_CHARACTER_、_NUMERIC_などの特別な名前のリストを使うこともできます。

- _ALL_を使うと、SASはSASデータセット内の同じ型(すべて数値または文字列)の変数をすべて使用します。
- _CHARACTER_を使うと、SASはデータセット内のすべての文字列変数を使用します。
- _NUMERIC_を使うと、SASはデータセット内のすべての数値変数を使用します。

この場合、_NUMERIC_キーワードを使うこともできます(次のプログラムの通り)。


Simple enough! Rather than you having to tell SAS how many variables and listing out exactly which ones you are grouping in an array, you can let SAS to the dirty work of counting the number of elements and listing the ones you include in your variable list. To do so, you simply define the dimension using an asterisk (*) and use the SAS list shortcut. You might find this strategy particularly helpful if you are grouping so many variables together into an array that you don't want to spend the time counting and listing them individually. Incidentally, throughout this lesson, we enclose the array's dimension (or index variable) in parentheses ( ). We could alternatively use braces { } or brackets [ ].  
The above program used a SAS list to shorten the list of variable names grouped into the fahr array. In some cases, you could also consider using the special name lists _ALL_, _CHARACTER_ and _NUMERIC_:  

* Use _ALL_ when you want SAS to use all of the same type of variables (all numeric or all character) in your SAS data set.
* Use _CHARACTER_ when you want SAS to use all of the character variables in your data set. 
* Use _NUMERIC_ when you want SAS to use all of the numeric variables in your data set.

In this case, we could have used the _NUMERIC_ keyword instead as shown in the following program.  

In [33]:
DATA avgfahrenheitthree;
    set avgcelsius;
    array fahr(*) _NUMERIC_;
    do i = 1 to 12;
            fahr(i) = 1.8*fahr(i) + 32;
    end;
RUN;
 
PROC PRINT data = avgfahrenheitthree;
    title 'Average Monthly Temperatures in Fahrenheit';
    id City;
    var jan feb mar apr may jun 
        jul aug sep oct nov dec;
RUN;

City,jan,feb,mar,apr,may,jun,jul,aug,sep,oct,nov,dec
"State College, PA",28.4,28.4,35.6,46.4,57.2,66.2,69.8,68.0,60.8,50.0,39.2,30.2
"Miami, FL",68.0,68.0,71.6,73.4,78.8,80.6,82.4,82.4,80.6,78.8,73.4,68.0
"St. Louis, MO",30.2,33.8,42.8,55.4,64.4,73.4,78.8,77.0,69.8,59.0,44.6,33.8
"New Orleans, LA",51.8,55.4,60.8,68.0,73.4,80.6,80.6,80.6,78.8,69.8,60.8,53.6
"Madison, WI",17.6,23.0,32.0,44.6,57.2,66.2,71.6,68.0,60.8,50.0,35.6,23.0
"Houston, TX",50.0,53.6,60.8,68.0,73.4,80.6,82.4,82.4,78.8,69.8,60.8,53.6
"Phoenix, AZ",53.6,57.2,60.8,69.8,78.8,87.8,91.4,89.6,86.0,73.4,60.8,53.6
"Seattle, WA",41.0,42.8,44.6,50.0,55.4,60.8,64.4,64.4,60.8,53.6,46.4,42.8
"San Francisco, CA",50.0,53.6,53.6,55.4,57.2,59.0,59.0,60.8,62.6,60.8,57.2,51.8
"San Diego, CA",55.4,57.2,59.0,60.8,62.6,66.2,69.8,71.6,69.8,66.2,60.8,57.2


### 新しい変数を配列ステートメントで作成する

これまでに、既存の変数をグループ化して配列を作る方法をいくつか学びました。しかし、ARRAY ステートメントから配列要素を省略すれば、新しい変数も作ることができます。ARRAYステートメントで既存の変数を参照しない場合、SASは自動的に新しい変数を作成し、デフォルトの名前を割り当てます。

#### 例

次のプログラムでは、再び10都市の月平均気温(Celsius)を月平均気温(Fahrenheit)に変換しています。既存のセルシウス温度jan、feb、...、decはcelsiusという配列にグループ化され、計算結果のFahrenheit温度はjanf、febf、...、decfという新しい変数に格納され、fahrという配列にグループ化されています。



### Creating New Variable in an Array Statement

So far, we have learned several ways to group existing variables into an array. We can also create new variables in an ARRAY statement by omitting the array elements from the statement. When our ARRAY statement fails to reference existing variables, SAS automatically creates new variables for us and assigns default names to them.

### Example

The following program again converts the average monthly Celsius temperatures in ten cities to average montly Fahrenheit temperatures. To do so, the already existing Celsius temperatures, jan, feb, ..., and dec, are grouped into an array called celsius, and the resulting Fahrenheit temperatures are stored in new variables janf, febf, ..., decf, which are grouped into an array called fahr:  

In [34]:
DATA avgtemps;
    set avgcelsius;
    array celsius(12) jan feb mar apr may jun 
                        jul aug sep oct nov dec;
    array fahr(12) janf febf marf aprf mayf junf
                    julf augf sepf octf novf decf;
    do i = 1 to 12;
            fahr(i) = 1.8*celsius(i) + 32;
    end;
RUN;
 
PROC PRINT data = avgtemps;
    title 'Average Monthly Temperatures';
    id City;
    var jan janf feb febf mar marf;
    var apr aprf may mayf jun junf;
    var jul julf aug augf sep sepf;
    var oct octf nov novf dec decf;
RUN;

City,jan,janf,feb,febf,mar,marf,apr,aprf,may,mayf,jun,junf,jul,julf,aug,augf,sep,sepf,oct,octf,nov,novf,dec,decf
"State College, PA",-2,28.4,-2,28.4,2,35.6,8,46.4,14,57.2,19,66.2,21,69.8,20,68.0,16,60.8,10,50.0,4,39.2,-1,30.2
"Miami, FL",20,68.0,20,68.0,22,71.6,23,73.4,26,78.8,27,80.6,28,82.4,28,82.4,27,80.6,26,78.8,23,73.4,20,68.0
"St. Louis, MO",-1,30.2,1,33.8,6,42.8,13,55.4,18,64.4,23,73.4,26,78.8,25,77.0,21,69.8,15,59.0,7,44.6,1,33.8
"New Orleans, LA",11,51.8,13,55.4,16,60.8,20,68.0,23,73.4,27,80.6,27,80.6,27,80.6,26,78.8,21,69.8,16,60.8,12,53.6
"Madison, WI",-8,17.6,-5,23.0,0,32.0,7,44.6,14,57.2,19,66.2,22,71.6,20,68.0,16,60.8,10,50.0,2,35.6,-5,23.0
"Houston, TX",10,50.0,12,53.6,16,60.8,20,68.0,23,73.4,27,80.6,28,82.4,28,82.4,26,78.8,21,69.8,16,60.8,12,53.6
"Phoenix, AZ",12,53.6,14,57.2,16,60.8,21,69.8,26,78.8,31,87.8,33,91.4,32,89.6,30,86.0,23,73.4,16,60.8,12,53.6
"Seattle, WA",5,41.0,6,42.8,7,44.6,10,50.0,13,55.4,16,60.8,18,64.4,18,64.4,16,60.8,12,53.6,8,46.4,6,42.8
"San Francisco, CA",10,50.0,12,53.6,12,53.6,13,55.4,14,57.2,15,59.0,15,59.0,16,60.8,17,62.6,16,60.8,14,57.2,11,51.8
"San Diego, CA",13,55.4,14,57.2,15,59.0,16,60.8,17,62.6,19,66.2,21,69.8,22,71.6,21,69.8,19,66.2,16,60.8,14,57.2


DATAステップはExample 7.6とよく似ています。違いは、Celsius温度を上書きするのではなく、計算したFahrenheit温度を新しい変数janf、febf、...、decfに保存する点です。最初のARRAYステートメントは、avgcelsiusデータセットのjan、feb、...、dec変数をcelsiusという1次元配列にグループ化するようSASに指示します。2つ目のARRAYステートメントは、SASに新しい変数janf、febf、...、decfを作成し、fahrという配列にグループ化するように指示しています。DOループはcelsius配列の12要素を処理し、Celsius温度をFahrenheit温度に変換して、結果をfahr配列に格納します。PRINTプロシージャは、12のCelsius温度と12のFahrenheit温度を横に並べて出力するようSASに指示します。SASプログラムを起動して実行し、PRINTプロシージャの出力を確認して、Celsius温度がFahrenheit温度に適切に変換されたことを確認してください。別の方法として、fahr配列の名前付けをSASに任せることもできます。  





The DATA step should look eerily similar to that of Example 7.6. The only thing that differs here is rather than writing over the Celsius temperatures, they are preserved by storing the calculated Fahrenheit temperatures in new variables called janf, febf, ..., and decf. The first ARRAY statement tells SAS to group the jan, feb, ..., dec variables in the avgcelsius data set into a one-dimensional array called celsius. The second ARRAY statement tells SAS to create twelve new variables called janf, febf, ..., and decf and to group them into an array called fahr. The DO loop processes through the twelve elements of the celsius array, converts the Celsius temperatures to Fahrenheit temperatures, and stores the results in the fahr array. The PRINT procedure then tells SAS to print the contents of the twelve Celsius temperatures and twelve Fahrenheit temperatures side-by-side. Launch and run the SAS program, and review the output from the PRINT procedure to convince yourself that the Celsius temperatures were properly converted to Fahrenheit temperatures.  
Alternatively, we could let SAS do the naming for us in the fahr array.  

In [35]:
DATA avgtempsinF;
    set avgcelsius;
    array celsius(12) jan feb mar apr may jun 
                      jul aug sep oct nov dec;
    array fahr(12);
    do i = 1 to 12;
            fahr(i) = 1.8*celsius(i) + 32;
    end;
RUN;
 
PROC PRINT data = avgtempsinF;
    title 'Average Monthly Temperatures in Fahrenheit';
    id City;
    var fahr1-fahr12;
RUN;

City,fahr1,fahr2,fahr3,fahr4,fahr5,fahr6,fahr7,fahr8,fahr9,fahr10,fahr11,fahr12
"State College, PA",28.4,28.4,35.6,46.4,57.2,66.2,69.8,68.0,60.8,50.0,39.2,30.2
"Miami, FL",68.0,68.0,71.6,73.4,78.8,80.6,82.4,82.4,80.6,78.8,73.4,68.0
"St. Louis, MO",30.2,33.8,42.8,55.4,64.4,73.4,78.8,77.0,69.8,59.0,44.6,33.8
"New Orleans, LA",51.8,55.4,60.8,68.0,73.4,80.6,80.6,80.6,78.8,69.8,60.8,53.6
"Madison, WI",17.6,23.0,32.0,44.6,57.2,66.2,71.6,68.0,60.8,50.0,35.6,23.0
"Houston, TX",50.0,53.6,60.8,68.0,73.4,80.6,82.4,82.4,78.8,69.8,60.8,53.6
"Phoenix, AZ",53.6,57.2,60.8,69.8,78.8,87.8,91.4,89.6,86.0,73.4,60.8,53.6
"Seattle, WA",41.0,42.8,44.6,50.0,55.4,60.8,64.4,64.4,60.8,53.6,46.4,42.8
"San Francisco, CA",50.0,53.6,53.6,55.4,57.2,59.0,59.0,60.8,62.6,60.8,57.2,51.8
"San Diego, CA",55.4,57.2,59.0,60.8,62.6,66.2,69.8,71.6,69.8,66.2,60.8,57.2



2つ目のARRAYステートメントでfahr配列を定義する際、fahr配列に含める要素の数(12)を指定しますが、配列にグループ化する変数は指定しません。これはSASに対して、i) 12個の新しい変数を作成すること、ii) 変数名の付け方はSASに任せること、の2点を示します。この場合、SASはデフォルトの名前として、配列名と1、2、3...配列の次元までの番号を連結したものを作成します。ここでは例えば、fahr1、fahr2、fahr3、...、fahr12という名前が作成されます。そのため、PRINTプロシージャのVARステートメントでは、Fahrenheit温度をfahr1からfahr12と参照しています。SASプログラムを起動して実行し、PRINTプロシージャの出力を確認して、Celsius温度がFahrenheit温度に適切に変換されたことを確認してください。



Note that when we define the fahr array in the second ARRAY statement, we specify how many elements the fahr array should contain (12), but we do not specify any variables to group into the array. That tells SAS two things: i) we want to create twelve new variables, and ii) we want to leave the naming of the variables to SAS. In this situation, SAS creates default names by concatenating the array name and the numbers 1, 2, 3, and so on, up to the array dimension. Here, for example, SAS creates the names fahr1, fahr2, fahr3, ..., up to fahr12. That's why we refer to the Fahrenheit temperatures as fahr1 to fahr12 in the PRINT procedure's VAR statement. Launch and run the SAS program, and review the output from the PRINT procedure to convince yourself that the Celsius temperatures were again properly converted to Fahrenheit temperatures.  

### 一時的な配列要素  

配列の要素が、DATAステップの間だけ必要な定数の場合、配列グループに関連付けられた変数を省略し、代わりに一時的な配列要素を使用できます。一時的な配列要素は変数のように振る舞いますが、以下の特徴があります。

* 結果のデータセットには現れない
* 名前がなく、配列名と次元でのみ参照できる
* 自動的に保持され、DATAステップの次の反復の開始時にミッシングにリセットされない

このセクションでは、Quality of Lifeデータのサブセットにエラーがないかどうかを確認する3つの例を見ていきます。最初の例では、10変数qul3a、qul3b、...、qul3jに記録されたデータが予想される範囲内にあるかを、配列を使わずに確認します。次に、同じ10変数に記録されたデータが予想される範囲内にあるかを、新しい変数error1、error2、error3に対応する配列を使って確認します。最後に、一時的な要素のみを含む配列を使って、同じ10変数に記録されたデータが予想される範囲内にあるかを確認します。

#### 例

次のプログラムでは、最初にQuality of Lifeデータのサブセット(変数qul3a、qul3b、...、qul3j)をqulというSASデータセットに読み込みます。次に、各変数の値がデータ形式から予想される1、2、3のいずれかとして記録されていることを確認します。変数の値が1、2、3以外の場合、その観測値はerrorsというデータセットに出力されます。それ以外の場合は、qulデータセットに出力されます。エラーチェックは配列を使わずに行われるため、プログラムには関係する10変数ごとに10個のif/thenステートメントがあります。



### Temporary Array Elements

When elements of an array are constants needed only for the duration of the DATA step, you can omit the variables associated with an array group and instead **use temporary array elements**. Although they behave like variables, temporary array elements:

* do not appear in the resulting data set;
* do not have names and can be only referenced by their array names and dimensions; and
* are automatically retained, rather than being reset to missing at the beginning of the next iteration of the DATA step.

In this section, we'll look at three examples that involve checking a subset of [Quality of Life](https://online.stat.psu.edu/onlinecourses/sites/stat481/files/lesson07/qul.v2.pdf) data for errors. In the following example, we'll look to see if the data recorded in ten variables — qul3a, qul3b, ..., and qul3j —are within an expected range without using an array. Then, we'll look to see if the data recorded in the same ten variables are within an expected range using an array that corresponds to three new variables error1, error2, and error3. Finally, we'll look to see if the data recorded in the same ten variables are within an expected range using an array containing only temporary elements.

### Example

The following program first reads a subset of [Quality of Life](https://online.stat.psu.edu/onlinecourses/sites/stat481/files/lesson07/qul.v2.pdf) data (variables qul3a, qul3b, ..., and qul3j) into a SAS data set called qul. Then, the program checks to make sure that the values for each variable have been recorded as either a 1, 2, or 3 as would be expected from the data form. If a value for one of the variables does not equal 1, 2, or 3, then that observation is output to a data set called errors. Otherwise, the observation is output to the qul data set. Because the error checking takes places **without using arrays** , the program contains a series of ten if/then statements, corresponding to each of the ten concerned variables:  

In [36]:
DATA qul errors;
    input subj qul3a qul3b qul3c qul3d qul3e 
               qul3f qul3g qul3h qul3i qul3j;
    flag = 0;
    if qul3a not in (1, 2, 3) then flag = 1;
    if qul3b not in (1, 2, 3) then flag = 1;
    if qul3c not in (1, 2, 3) then flag = 1;
    if qul3d not in (1, 2, 3) then flag = 1;
    if qul3e not in (1, 2, 3) then flag = 1;
    if qul3f not in (1, 2, 3) then flag = 1;
    if qul3g not in (1, 2, 3) then flag = 1;
    if qul3h not in (1, 2, 3) then flag = 1;
    if qul3i not in (1, 2, 3) then flag = 1;
    if qul3j not in (1, 2, 3) then flag = 1;
    if flag = 1 then output errors;
                else output qul;
    drop flag;
    DATALINES;
    110011 1 2 3 3 3 3 2 1 1 3
    210012 2 3 4 1 2 2 3 3 1 1
    211011 1 2 3 2 1 2 3 2 1 3
    310017 1 2 3 3 3 3 3 2 2 1
    411020 4 3 3 3 3 2 2 2 2 2
    510001 1 1 1 1 1 1 2 1 2 2
    ;
RUN;
PROC PRINT data = qul;
    TITLE 'Observations in Qul data set with no errors';
RUN;
PROC PRINT data = errors;
    TITLE 'Observations in Qul data set with errors';
RUN;

Obs,subj,qul3a,qul3b,qul3c,qul3d,qul3e,qul3f,qul3g,qul3h,qul3i,qul3j
1,110011,1,2,3,3,3,3,2,1,1,3
2,211011,1,2,3,2,1,2,3,2,1,3
3,310017,1,2,3,3,3,3,3,2,2,1
4,510001,1,1,1,1,1,1,2,1,2,2
Obs,subj,qul3a,qul3b,qul3c,qul3d,qul3e,qul3f,qul3g,qul3h,qul3i,qul3j
1,210012,2,3,4,1,2,2,3,3,1,1
2,411020,4,3,3,3,3,2,2,2,2,2


INPUTステートメントで最初に、1人の被験者のQuality of Lifeデータの観測値を読み込みます。観測値はエラーがない(フラグは最初0に設定)と見なされますが、10個の値のいずれかが範囲外であれば、エラーありと見なされます(フラグが1に設定)。観測値がエラーを含む(フラグ=1)と判断された場合、errorsデータセットに出力されます。そうでない場合(フラグ=0)、qulデータセットに出力されます。

入力データセットの2つの観測値にデータ記録エラーがあることに注意してください。被験者210012のqul3c値が4と記録され、被験者411020のqul3a値も4と記録されています。次にSASプログラムを起動して実行します。qulには4つのクリーンデータ観測値が含まれ、errorsには2つの不正データ観測値が含まれていることを確認してください。

また、このようなケースは典型的に配列を使うべき状況であることにも注目してください。まだ納得がいかない場合は、100個程度の変数についてエラーをチェックするためのif/thenステートメントを書く必要があると想像してみてください。

次のプログラムでは、前のプログラムと同じエラーチェックを行いますが、ここではboundsとquldataの2つの配列を使用して実行されます。  




The INPUT statement first reads an observation of data containing one subject's quality of life data. An observation is assumed to be error-free (flag is initially set to 0) until it is found to be in error (flag is set to 1 if any of the ten values are out of range). If an observation is deemed to contain an error (flag = 1) after looking at each of the ten values, it is output to the errors data set. Otherwise (flag = 0) , it is output to the qul data set.  
First, note that two of the observations in the input data set contain data recording errors. The qul3c value for subject 210012 was recorded as 4, as was the qul3a value for subject 411020. Then, launch and run the SAS program. Review the output to convince yourself that qul contains the four observations with clean data, and errors contains the two observations with bad data.  
You should also appreciate that this is a classic situation that cries out for using arrays. If you aren't yet convinced, imagine how long the above program would be if you had to write similar if/then statements to check for errors in, say, a hundred such variables.  
The following program performs the same error checking as the previous program except here the error checking is accomplished **using two arrays** , bounds and quldata:  

In [37]:
DATA qul errors;
    input subj qul3a qul3b qul3c qul3d qul3e 
            qul3f qul3g qul3h qul3i qul3j;
    array bounds (3) error1 - error3 (1 2 3);
    array quldata (10) qul3a -- qul3j;
    flag = 0;
    do i = 1 to 10;
        if quldata(i) ne bounds(1) and
            quldata(i) ne bounds(2) and
            quldata(i) ne bounds(3)
        then flag = 1;
    end;
    if flag = 1 then output errors;
                else output qul;
    drop i flag;
    DATALINES;
    110011 1 2 3 3 3 3 2 1 1 3
    210012 2 3 4 1 2 2 3 3 1 1
    211011 1 2 3 2 1 2 3 2 1 3
    310017 1 2 3 3 3 3 3 2 2 1
    411020 4 3 3 3 3 2 2 2 2 2
    510001 1 1 1 1 1 1 2 1 2 2
    ;
RUN;
 
PROC PRINT data = qul;
    TITLE 'Observations in Qul data set with no errors';
RUN;
 
PROC PRINT data = errors;
    TITLE '
Observations in Qul data set with errors';
RUN;

Obs,subj,qul3a,qul3b,qul3c,qul3d,qul3e,qul3f,qul3g,qul3h,qul3i,qul3j,error1,error2,error3
1,110011,1,2,3,3,3,3,2,1,1,3,1,2,3
2,211011,1,2,3,2,1,2,3,2,1,3,1,2,3
3,310017,1,2,3,3,3,3,3,2,2,1,1,2,3
4,510001,1,1,1,1,1,1,2,1,2,2,1,2,3
Obs,subj,qul3a,qul3b,qul3c,qul3d,qul3e,qul3f,qul3g,qul3h,qul3i,qul3j,error1,error2,error3
1,210012,2,3,4,1,2,2,3,3,1,1,1,2,3
2,411020,4,3,3,3,3,2,2,2,2,2,1,2,3



前のプログラムとこのプログラムを比べると、違いは2つのARRAY定義ステートメントと、エラーチェックを行う反復DOループ内のIF/THENステートメントの存在のみです。

最初のARRAYステートメントでは、番号範囲リストを使ってboundsという配列を定義し、新しい変数error1、error2、error3の3つを含みます。変数リストerror1-error3の後ろの"(1 2 3)"は、SASにbounds配列の要素を1、2、3に初期化するように指示しています。一般に、この方法で配列を初期化します。つまり、配列の要素数と同じ数値をリストアップし、各値の間にスペースを入れます。配列に文字定数を含める場合は、値をシングルクォートで囲む必要があります。例えば、次のARRAYステートメントは、SASに文字列配列(ドル記号$)weekdaysを定義し、要素をM、T、W、R、Fに初期化するよう指示します。
ARRAY weekdays(5) $ ('M' 'T' 'W' 'R' 'F');

2つ目のARRAYステートメントでは、名前範囲リストを使ってquldataという配列を定義し、10個のQuality of Life変数を含みます。IF/THENステートメントは、前のプログラムとは少し異なるロジックを使い、quldata配列の要素とbounds配列の要素を比較して、値が範囲外かどうかをSASに確認させています。

次に、SASプログラムを起動して実行します。前と同様に、qulには4つのクリーンデータ観測値が含まれ、errorsには2つの不正データ観測値が含まれていることを、出力を確認して確認してください。また、新しい変数error1、error2、error3がデータセットに残っていることにも注目してください。

前のプログラムでは、有効値1、2、3が一時的にしか必要ありません。したがって、boundsの配列を定義する際に、一時的な配列要素を使うこともできました。次のプログラムはそうしたもので、前のプログラムと同じですが、boundsの配列を定義する際に、新しい変数error1、error2、error3ではなく、一時的な配列要素を使っている点が異なります。



If you compare this program to the previous program, you'll see that the only differences here are the presence of two ARRAY definition statements and the IF/THEN statement within the iterative DO loop that does the error checking.  
The first ARRAY statement uses a numbered range list to define an array called bounds that contains three new variables — error1, error2, and error3. The "(1 2 3)" that appears after the variable list error1-error3 tells SAS to set, or initialize, the elements of the array to equal 1, 2, and 3. In general, you initialize an array in this manner, namely listing as many values as their are elements of the array and separating each pair of values with a space. If you intend for your array to contain character constants, you must put the values in single quotes. For example, the following ARRAY statement tells SAS to define a character array (hence the dollar sign \$) called weekdays:  

```
ARRAY weekdays(5) $ ('M' 'T' 'W' 'R' 'F');
```

and to initialize the elements of the array as M, T, W, R, and F.  
The second ARRAY statement uses a name range list to define an array called quldata that contains the ten quality of life variables. The IF/THEN statement uses slightly different logic than the previous program to tell SAS to compare the elements of the quldata array to the elements of the bounds array to determine whether any of the values are out of range.  
Now, launch and run the SAS program. Review the output to convince yourself that just as before qul contains the four observations with clean data, and errors contains the two observations with bad data. Also, note that the three new error variables error1, error2, and error3 remain present in the data set.  
The valid values 1, 2, and 3 are needed only temporarily in the previous program. Therefore, we alternatively could have used temporary array elements in defining the bounds array. The following program does just that. It is identical to the previous program except here the bounds array is defined **using temporary array elements** rather than using three new variables error1, error2, and error3:  

In [38]:
DATA qul errors;
    input subj qul3a qul3b qul3c qul3d qul3e 
            qul3f qul3g qul3h qul3i qul3j;
    array bounds (3) _TEMPORARY_ (1 2 3);
    array quldata (10) qul3a -- qul3j;
    flag = 0;
    do i = 1 to 10;
        if quldata(i) ne bounds(1) and
            quldata(i) ne bounds(2) and
            quldata(i) ne bounds(3)
        then flag = 1;
    end;
    if flag = 1 then output errors;
                else output qul;
    drop i flag;
    DATALINES;
    110011 1 2 3 3 3 3 2 1 1 3
    210012 2 3 4 1 2 2 3 3 1 1
    211011 1 2 3 2 1 2 3 2 1 3
    310017 1 2 3 3 3 3 3 2 2 1
    411020 4 3 3 3 3 2 2 2 2 2
    510001 1 1 1 1 1 1 2 1 2 2
    ;
RUN;
 
PROC PRINT data = qul;
    TITLE 'Observations in Qul data set with no errors';
RUN;
 
PROC PRINT data = errors;
    TITLE '
Observations in Qul data set with errors';
RUN

Obs,subj,qul3a,qul3b,qul3c,qul3d,qul3e,qul3f,qul3g,qul3h,qul3i,qul3j
1,110011,1,2,3,3,3,3,2,1,1,3
2,211011,1,2,3,2,1,2,3,2,1,3
3,310017,1,2,3,3,3,3,3,2,2,1
4,510001,1,1,1,1,1,1,2,1,2,2
Obs,subj,qul3a,qul3b,qul3c,qul3d,qul3e,qul3f,qul3g,qul3h,qul3i,qul3j
1,210012,2,3,4,1,2,2,3,3,1,1
2,411020,4,3,3,3,3,2,2,2,2,2


前のプログラムとこのプログラムを比べると、違いはbounds配列の定義に_TEMPORARY_引数が存在する点のみです。bounds配列は再び有効値"(1 2 3)"で初期化されています。

SASプログラムを起動して実行します。前と同様に、qulには4つのクリーンデータ観測値が含まれ、errorsには2つの不正データ観測値が含まれていることを、出力を確認して確認してください。また、一時的な配列要素がデータセットに現れないことにも注目してください。

If you compare this program to the previous program, you'll see that the only difference here is the presence of the **_TEMPORARY_** argument in the definition of the bounds array. The bounds array is again initialized to the three valid values "(1 2 3)".  
Launch and run the SAS program. Review the output to convince yourself that just as before qul contains the four observations with clean data, and errors contains the two observations with bad data. Also, note that the temporary array elements do not appear in the data set.  

### 配列の境界

これまで検討してきた配列はそれぞれデフォルトで下限1と次元の要素数に等しい上限が定義されています。例えば、配列my_arrayは

`ARRAY my_array(4) el1 el2 el3 el4;`

下限1、上限4を持っています。このセクションでは、配列の境界に関連する3つの例を見ていきます。最初の例では、DIM関数を使ってDO ループのインデックス変数の上限を動的に変更します(あらかじめ指定するのではなく)。2番目の例では、1次元配列の下限と上限を定義して、境界付き配列を作成します。3番目の例では、LBOUND関数とHBOUND関数を使って、DO ループのインデックス変数の下限と上限を動的に変更します。

例
次のプログラムは、5人の被験者の6つの調査質問(q1、q2、...、q6)への yes/no 回答をサーベイという一時的な SAS データセットに読み込みます。yes の回答は 2 としてコード化され、no の回答は 1 としてコード化されます。変数 q3、q4、q5、q6 の 4 つだけが qxs という 1 次元配列に格納されます。次に、DO LOOP と DIM 関数を組み合わせて、4 つの変数の応答を再コード化し、2 を 1 に、1 を 0 に変更します。




### Array Bounds

Each of the arrays we've considered thus far have been defined, by default, to have a lower bound of 1 and an upper bound which equals the number of elements in the array's dimension. For example, the array my_array:

`ARRAY my_array(4) el1 el2 el3 el4;`

has a lower bound of 1 and an upper bound of 4. In this section, we'll look at three examples that concern the bounds of an array. In the first example, we'll use the DIM function to change the upper bound of a DO loop's index variable dynamically (rather than stating it in advance). In the second example, we'll define the lower and upper bounds of a one-dimensional array to create a bounded array. In the third example, we'll use the LBOUND and HBOUND functions to change the lower and upper bounds of a DO loop's index variable dynamically.

### Example

The following program reads the yes/no responses of five subjects to six survey questions (q1, q2, ..., q6) into a temporary SAS data set called survey. A yes response is coded and entered as a 2, while a no response is coded and entered as a 1. Just four of the variables (q3, q4, q5, and q6) are stored in a one-dimensional array called qxs. Then, a DO LOOP, in conjunction with the DIM function, is used to recode the responses to the four variables so that a 2 is changed to a 1, and a 1 is changed to a 0:  

In [39]:
DATA survey (DROP = i);
    INPUT subj q1 q2 q3 q4 q5 q6;
    ARRAY qxs(4) q3-q6;
    DO i = 1 to dim(qxs);
        qxs(i) = qxs(i) - 1;
    END;
    DATALINES;
1001 1 2 1 2 1 1
1002 2 1 2 2 2 1
1003 2 2 2 1 . 2
1004 1 . 1 1 1 2
1005 2 1 2 2 2 1
;
RUN;
 
PROC PRINT data = survey;
    TITLE 'The survey data using dim function';
RUN;

Obs,subj,q1,q2,q3,q4,q5,q6
1,1001,1,2,0,1,0,0
2,1002,2,1,1,1,1,0
3,1003,2,2,1,0,.,1
4,1004,1,.,0,0,0,1
5,1005,2,1,1,1,1,0


最初に注目すべきは、すべてのサーベイ変数(q1、...、q6)がサーベイデータセットに読み込まれるものの、ARRAY ステートメントでは変数の 4 つ(q3、q4、q5、q6)のみが 1 次元配列 qxs にグループ化されている点です。例えば、qxs(1) は変数 q3 に、qxs(2) は変数 q4 に対応しています。次に、配列を要素 1 から要素 4 まで処理するように SAS に指示するのではなく、DO ループでは SAS に DIM(qxs)までの要素を処理するように指示しています。一般に、DIM 関数は配列の要素数を返します。この場合は 4 です。DO ループは SAS に値を 1 引いて再コード化するように指示しています。そしてインデックス変数 i がサーベイデータセットにデフォルトで出力されるため、DROP されます。



First, note that although all of the survey variables (q1, ..., q6) are read into the survey data set, the ARRAY statement groups only 4 of the variables (q3, q4, q5, q6) into the one-dimensional array qxs. For example, qxs(1) corresponds to the q3 variable, qxs(2) corresponds to the q4 variable, and so on. Then, rather than telling SAS to process the array from element 1 to element 4, the DO loop tells SAS to process the array from element 1 to the more general DIM(qxs). In general, the DIM function returns the number of the elements in the array, which in this case is 4. The DO loop tells SAS to recode the values by simply subtracting 1 from each value. And, the index variable i is output to the survey data set by default and is therefore dropped.

例
前に説明および例示したように、SAS に配列の下限を特に指定しない場合、SAS は下限を 1 と見なします。ほとんどの配列では、下限 1 と要素数が上限というのが便利なので、通常は上限と下限の両方を指定する必要はありません。しかし、より便利な場合は、任意の次元の上限と下限の両方を変更することができます。

前の例では、配列要素 qxs(1) が変数 q3 に、qxs(2) が q4 に対応するのは少し面倒かもしれません。qxs(3) が q3 に、qxs(4) が q4 に対応する方が分かりやすいかもしれません。次のプログラムは機能的には前のプログラムと同じですが、ここでは qxs 配列の下限を 3、上限を 6 と定義することで再コード化を実現しています。



### Example

As previously discussed and illustrated, if you do not specifically tell SAS the lower bound of an array, SAS assumes that the lower bound is 1. For most arrays, 1 is a convenient lower bound and the number of elements is a convenient upper bound, so you usually don't need to specify both the lower and upper bounds. However, in cases where it is more convenient, you can modify both bounds for any array dimension.  
In the previous example, perhaps you find it a little awkward that the array element qxs(1) corresponds to the q3 variable, the array element qxs(2) corresponds to the q4 variable, and so on. Perhaps you would find it more clear for the array element qxs(3) to correspond to the q3 variable, the array element qxs(4) to correspond to the q4 variable, ..., and the array element qxs(6) to correspond to the q6 variable. The following program is similar in function to the previous program, except here the task of recoding is accomplished by defining the lower bound of the qxs array to be 3 and the upper bound to be 6:  

In [40]:
DATA survey2 (DROP = i);
    INPUT subj q1 q2 q3 q4 q5 q6;
    ARRAY qxs(3:6) q3-q6;
    DO i = 3 to 6;
        qxs(i) = qxs(i) - 1;
    END;
    DATALINES;
1001 1 2 1 2 1 1
1002 2 1 2 2 2 1
1003 2 2 2 1 . 2
1004 1 . 1 1 1 2
1005 2 1 2 2 2 1
;
RUN;
 
PROC PRINT data = survey2;
    TITLE 'The survey data using bounded arrays';
RUN;

Obs,subj,q1,q2,q3,q4,q5,q6
1,1001,1,2,0,1,0,0
2,1002,2,1,1,1,1,0
3,1003,2,2,1,0,.,1
4,1004,1,.,0,0,0,1
5,1005,2,1,1,1,1,0


この プログラムと前のプログラムを比較すると、2 つの違いがあることがわかります。1 つ目は、ARRAY ステートメントでここでは qxs 配列の下限を 3、上限を 6 と定義している点です。一般に、この方法で任意の配列次元の下限と上限を定義できます。つまり、下限を指定し、その後にコロン(:)、そして上限を指定します。2 つ目の違いは、DO ループでは、インデックス変数 i の範囲が明示的に 3 から 6 まで定義されている点です。これは前のプログラムの 1 から DIM(qxs) (この場合は 4) までとは異なります。



If you compare this program with the previous program, you'll see that only two things differ. The first difference is that the ARRAY statement here defines the lower bound of the qxs array to be 3 and the upper bound to be 6. In general, you can always define the lower and upper bounds of any array dimension in this way, namely by specifying the lower bound, then a colon (:), and then the upper bound. The second difference is that, for the DO loop, the bounds on the index variable i are specifically defined here to be between 3 and 6 rather than 1 to DIM(qxs) (which in this case is 4).  

例
さらに、配列次元の境界の扱いを自動化することができます。次のプログラムでは、前の 2 つのプログラムと同様に、1 次元配列 qxs を使って 4 つの調査変数を再コード化しています。しかし、ここではアスタリスク(*) を使って SAS に qxs 配列の次元を判断させ、LBOUND 関数と HBOUND 関数を使って、DO ループのインデックス変数の下限と上限をそれぞれ動的に指定しています。 




### Example

Now, there's still a little bit more that we can do to automate the handling of the bounds of an array dimension. The following program again uses a one-dimensional array qxs to recode four survey variables as did the previous two programs. Here, though, an asterisk (*) is used to tell SAS to determine the dimension of the qxs array, and the LBOUND and HBOUND functions are used to tell SAS to determine, respectively, the lower and upper bounds of the DO loop's index variable dynamically:  

In [41]:
DATA survey3 (DROP = i);
    INPUT subj q1 q2 q3 q4 q5 q6;
    ARRAY qxs(*) q3-q6;
    DO i = lbound(qxs) to hbound(qxs);
        qxs(i) = qxs(i) - 1;
    END;
    DATALINES;
1001 1 2 1 2 1 1
1002 2 1 2 2 2 1
1003 2 2 2 1 . 2
1004 1 . 1 1 1 2
1005 2 1 2 2 2 1
;
RUN;
 
PROC PRINT data = survey3;
    TITLE 'The survey data by changing upper and lower bounds automatically';
RUN;

Obs,subj,q1,q2,q3,q4,q5,q6
1,1001,1,2,0,1,0,0
2,1002,2,1,1,1,1,0
3,1003,2,2,1,0,.,1
4,1004,1,.,0,0,0,1
5,1005,2,1,1,1,1,0


このプログラムと前のプログラムを比較すると、2 つの違いがあります。1 つ目は、ARRAY ステートメントの中のアスタリスク (*) が、qxs の宣言時に配列の次元を SAS に判断させることを意味する点です。SAS は配列の要素数をカウントし、qxs の次元は 4 と判断します。2 つ目は、DO ループでは、インデックス変数 i の範囲が LBOUND(qxs) から HBOUND(qxs) まで動的に決定されている点です。



If you compare this program with the previous program, you'll see that only two things differ. The first difference is that the asterisk (*) that appears in the the ARRAY statement tells SAS to determine the bounds on the dimensions of the array during the declaration of qxs. SAS counts the number of elements in the array and determines that the dimension of qxs is 4. The second difference is that, for the DO loop, the bounds on the index variable i are determined dynamically to be between LBOUND(qxs) and HBOUND(qxs).  

### 2次元配列  

2次元配列は、1次元配列の直接的な拡張です。次のような1次元配列

`ARRAY barkers(4) dog1-dog4;` 

を変数の1行

`dog1 dog2 dog3 dog4`

と考えることができます。一方、次のような2次元配列

`ARRAY pets(2,4) dog1-dog4 cat1-cat4;`

は、変数の複数行として考えられます。

dog1 dog2 dog3 dog4  
cat1 cat2 cat3 cat4  

上の ARRAY ステートメントが示すように、2 次元配列を定義するには、カンマ区切りで各次元の要素数を指定します。一般に、最初の次元数が配列の行数を、2 番目の次元数が列数を示します。

2次元配列を定義すると、配列要素は ARRAY ステートメントの順序でグループ化されます。例えば、配列 horse を

`ARRAY horse(3,5) x1-x15;`

と定義すると、SAS は以下のように要素を割り当てます。

x1 x2 x3 x4 x5  
x6 x7 x8 x9 x10
x11 x12 x13 x14 x15


このプログラムは、家族歴データのサブセットで欠損値を検索します。ここでは、edit という 2 次元配列を 1 つ使用しています。最初の行には実際のデータが入り、2 番目の行には、2 次元配列の対応する列の観測データについての欠損の有無を示す 0/1 のインジケータが入ります。

プログラムには ARRAY ステートメントが 1 つあり、2 行 14 列の 2 次元配列 edit を定義しています。ARRAY ステートメントは、SAS に家族歴変数(fhx1、...、fhx14)を最初の次元にグループ化し、ステータス変数(stat1、...、stat14)を 2 番目の次元にグループ化するよう指示しています。次に、DO ループは SAS に 14 個の変数の内容を確認し、ステータス次元の各要素に値 0("edit(2,i) = 0;")を割り当てるよう指示しています。ただし、edit 次元の要素が欠損値の場合は、SAS にステータス次元の要素を 0 から 1 に変更するよう指示しています("if edit(1,i) = . then edit(2,i) = 1")。


### Two-Dimensional Arrays

Two-dimensional arrays are straightforward extensions of one-dimensional arrays. You can think of one-dimensional arrays such as the array barkers:

`ARRAY barkers(4) dog1-dog4;`

as a single row of variables:

`dog1 dog2 dog3 dog4`

And two-dimensional arrays such as the array pets:

`ARRAY pets(2,4) dog1-dog4 cat1-cat4;`

as multiple rows of variables:

```
dog1  dog2  dog3  dog4
cat1  cat2  cat3  cat4
```

As the previous ARRAY statement suggests, to define a two-dimensional array, you specify the number of elements in each dimension, separated by a comma. In general, the first dimension number tells SAS how many rows your array needs, while the second dimension number tells SAS how many columns your array needs.

When you define a two-dimensional array, the array elements are grouped in the order in which they appear in the ARRAY statement. For example, SAS assigns the elements of the array horse:

`ARRAY horse(3,5) x1-x15;`

as follows:

```
x1  x2  x3  x4  x5
x6  x7  x8  x9  x10
x11 x12 x13 x14 x15
```

In this section, we'll look at two examples that involve checking a subset of [Family History](https://online.stat.psu.edu/onlinecourses/sites/stat481/files/lesson07/fhx.v2.pdf) data for missing values. We'll use one two-dimensional array — the first dimension to store the actual data and the second dimension to store binary status variables that indicate whether a particular data value is missing or not.

### Example

This program searches a subset of the family history data for missing values. Here, we use one two-dimensional array called edit. The first row contains the actual data and the second row contains a 0/1 indicator of missingness for the observed data in the corresponding column of the two-dimensional array.  

In [43]:
DATA fhx;
    input subj v_date mmddyy8. fhx1-fhx14;
    array edit(2,14) fhx1-fhx14 stat1-stat14;
    do i = 1 to 14;
        edit(2,i) = 0;
        if edit(1,i) = . then edit(2,i) = 1;
    end;
    DATALINES;
220004  07/27/93  0  0  0  .  8  0  0  1  1  1  .  1  0  1
410020  11/11/93  0  0  0  .  0  0  0  0  0  0  .  0  0  0
520013  10/29/93  0  0  0  .  0  0  0  0  0  0  .  0  0  1
520068  08/10/95  0  0  0  0  0  1  1  0  0  1  1  0  1  0
520076  08/25/95  0  0  0  0  1  8  0  0  0  1  1  0  0  1
;
RUN;
 
PROC PRINT data = fhx;
    var fhx1-fhx14;
    TITLE 'The FHX data itself';
RUN;
 
PROC PRINT data = fhx;
    var stat1-stat14;
    TITLE 'The presence of missing values in FHX data';
RUN;

Obs,fhx1,fhx2,fhx3,fhx4,fhx5,fhx6,fhx7,fhx8,fhx9,fhx10,fhx11,fhx12,fhx13,fhx14
1,3,0,0,0,.,8,0,0,1,1,1,.,1,0
2,3,0,0,0,.,0,0,0,0,0,0,.,0,0
3,3,0,0,0,.,0,0,0,0,0,0,.,0,0
4,5,0,0,0,0,0,1,1,0,0,1,1,0,1
5,5,0,0,0,0,1,8,0,0,0,1,1,0,0
Obs,stat1,stat2,stat3,stat4,stat5,stat6,stat7,stat8,stat9,stat10,stat11,stat12,stat13,stat14
1,0,0,0,0,1,0,0,0,0,0,0,1,0,0
2,0,0,0,0,1,0,0,0,0,0,0,1,0,0
3,0,0,0,0,1,0,0,0,0,0,0,1,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0


We have just one ARRAY statement that defines the two-dimensional array edit containing 2 rows and 14 columns. The ARRAY statement tells SAS to group the family history variables (fhx1, ..., fhx14) into the first dimension and to group the status variables (stat1, ..., stat14) into the second dimension. Then, the DO loop tells SAS to review the contents of the 14 variables and to assign each element of the status dimension a value of 0 ("edit(2,i) = 0;"). If the element of the edit dimension is missing, however, then SAS is told to change the element of the status dimension from a 0 to a 1 ("if edit(1,i) = . then edit(2,i) = 1").  

## データの再形成

### 広い/長いデータとは?

広いとは、複数の列が1つの観測値に対応していることを意味します。例えば、visit1、visit2、visit3などです。


## Reshaping Data

What is wide/long data?

Wide means that we have multiple columns per observations, e.g. visit1, visit2, visit3

In [44]:
DATA wide;
    INPUT id visit1 visit2 visit3;
    INFILE DATALINES;
    DATALINES;
1 10 4 3
2 5 6 .
;
RUN;

PROC PRINT data = wide;
   TITLE 'Wide Dataset';
RUN;

Obs,id,visit1,visit2,visit3
1,1,10,4,3
2,2,5,6,.


長いとは、1つの観測値に対して複数の行があることを意味します。  



Long means that we have multiple rows per observation.

In [46]:
DATA long;
    INPUT id visit value;
    INFILE DATALINES;
    DATALINES;
1 1 10 
1 2 4 
1 3 3
2 1 5 
2 2 6 
2 3 .
;
RUN;

PROC PRINT data = long;
   TITLE 'Long Dataset';
RUN;

Obs,id,visit,value
1,1,1,10
2,1,2,4
3,1,3,3
4,2,1,5
5,2,2,6
6,2,3,.


SASでは、広い形式と長い形式の間でデータを再形成する方法が2つあります。

* PROC TRANSPOSE
* DATAステップ

いくつかの例を使ってこの両方の方法を探ります。

### 長い(縦長の)データから広い(横長の)データへの再形成

前のセクションの tallgradesデータセットを思い出してください。

In SAS, there are two ways to reshape data between wide and long formats

* PROC TRANSPOSE
* DATA step

We will explore both ways with some examples.

### Reshaping Data from Long (Tall) Wide (Fat)

Recall the tallgrades dataset from an earlier section.

In [12]:
DATA tallgrades;
    input idno 1-2 l_name $ 5-9 gtype $ 12-13 grade 15-17;
    cards;
10  Smith  E1  78
10  Smith  E2  82
10  Smith  E3  86
10  Smith  E4  69
10  Smith  P1  97
10  Smith  F1 160
11  Simon  E1  88
11  Simon  E2  72
11  Simon  E3  86
11  Simon  E4  99
11  Simon  P1 100
11  Simon  F1 170
12  Jones  E1  98
12  Jones  E2  92
12  Jones  E3  92
12  Jones  E4  99
12  Jones  P1  99
12  Jones  F1 185
;
RUN;
 
PROC PRINT data = tallgrades NOOBS;
   TITLE 'The tall grades data set';
RUN;

idno,l_name,gtype,grade
10,Smith,E1,78
10,Smith,E2,82
10,Smith,E3,86
10,Smith,E4,69
10,Smith,P1,97
10,Smith,F1,160
11,Simon,E1,88
11,Simon,E2,72
11,Simon,E3,86
11,Simon,E4,99



tallgradesデータセットには、各学生の各成績が1行ずつ含まれています。学生は、IDナンバー(idno)と苗字(l_name)で識別されています。このデータセットには6種類の成績が含まれています。それぞれ100点満点の試験1(E1)、試験2(E2)、試験3(E3)、試験4(E4)、100点満点のプロジェクト1(P1)、および200点満点の期末試験(F1)です。SASプログラムを起動して実行し、次の2つの例でtallgradesデータセットを操作できるようにしましょう。




The tallgrades data set contains one observation for each grade for each student. Students are identified by their id number (idno) and last name (l_name). The data set contains six different types of grades: exam 1 (E1), exam 2 (E2), exam 3 (E3), exam 4 (E4), each worth 100 points; one project (P1) worth 100 points; and a final exam (F1) worth 200 points. Launch and run the SAS program so that we can work with the tallgrades data set in the next two examples.

例1
この例では、DATAステップを使ってtallgradesデータセットを長い形式から広い形式に転置します。これには、配列、RETAINステートメント、OUTPUTステートメント、そしてFIRST.およびLAST.SAS変数の使用が必要になります。

### Example

In this example, we will transpose the tallgrades dataset from long to wide format by using a DATA step. This will require the use of an array and both the RETAIN and OUTPUT statements, and the FIRST. and LAST. SAS variables.  

In [13]:
DATA fatgrades;
   set tallgrades;
   by idno;
   retain E1-E4 P1 F1 i;
   array allgrades (6) E1-E4 P1 F1;
   if first.idno then i = 1;
   allgrades(i) = grade;
   if last.idno then output;
   i = i + 1;
   drop i gtype grade;
RUN;
 
PROC PRINT data=fatgrades;
  title 'The fat grades data set';
RUN;

Obs,idno,l_name,E1,E2,E3,E4,P1,F1
1,10,Smith,78,82,86,69,97,160
2,11,Simon,88,72,86,99,100,170
3,12,Jones,98,92,92,99,99,185



この長いコードは怖いですね!少し分解しましょう。まず、tallgradesデータセットはidnoごとに処理されます。これにより、first.idnoとlast.idno変数を使用できるようになります。ARRAYステートメントは、allgradesという名前の配列を定義し、番号付きの範囲リストを使って、この配列を6つの(初期化されていない)変数E1、E2、E3、E4、P1、F1に関連付けています。allgrades配列は、転置される前に、各学生の6つの成績を保持するために使用されます。

配列の要素は、インデックス変数を使って割り当てられなければならないため、この転置は次のように行われます:

("if first.idno then i = 1;") 入力観測値に、まだデータセットで出会っていない学生idnoが含まれている場合、インデックス変数iは1に初期化されます。入力観測値に新しい学生idnoが含まれていない場合は、何もせずに次のステップに進みます。

("allgrades(i) = grade;") 現在の観測値からの成績が、配列allgradesに割り当てられます。(例えば、入力観測値がSmithの最初の成績である場合、allgrades(1)に値78が割り当てられます。入力観測値がSmithの2番目の成績である場合、allgrades(2)に値82が割り当てられます。以下同様です。) これは、各学生の成績の順序が同じであることを前提としています。

("if last.idno then output;") 入力観測値がそのidnoを含む最後の観測値である場合、プログラムデータベクトル(allgradesを含む)を出力データセットに出力します。(例えば、入力観測値がSmithの期末試験の成績である場合、彼の6つの成績を含む横長の観測値を出力します)。入力観測値がそのidnoを含む最後の観測値でない場合は、何もせずに次のステップに進みます。  

("i = i + 1;") 次に、インデックス変数iを1増やします。(例えば、iが1の場合、2に変更します)。

("retain E1-E4 P1 F1 i;") 次の反復の開始時にE1、E2、E3、E4、P1、F1、iをミッシングに設定するのではなく、現在の値を保持します。(例えば、Smithの場合、allgrades(1)は値78を保持し、allgrades(2)は値82を保持します。など) インデックスカウンタiについても同様です。新しい生徒に出会うまで、この変数をリセットしたくありません。

プログラムは、データセットの最後の観測値に出会うまで、上記の5ステップを繰り返し行います。その後、変数i、gtype、gradeは出力のfatgradesデータセットから削除されます。



Yikes! This code looks scary! Let's dissect it a bit. First, the tallgrades data set is processed BY idno. Doing so, makes the first.idno and last.idno variables available for us to use. The ARRAY statement defines an array called allgrades and, using a numbered range list, associates the array with six (uninitialized) variables E1, E2, E2, E4, P1, and F1. The allgrades array is used to hold the six grades for each student before they are output in their transposed direction to the fatgrades data set. Because the elements of any array, and therefore allgrades, must be assigned using an index variable, this is how the transposition takes place:  

* ("if first.idno then i = 1;") If the input observation contains a student idno that hasn't yet been encountered in the data set, then the index variable i is initialized to 1. If the input observation doesn't contain a new student idno, then do nothing other than advance to the next step.
* ("allgrades(i) = grade;") The grade from the current observation is assigned to the array allgrades. (For example, if the input observation contains Smith's first grade, then allgrades(1) is assigned the value 78. If the input observation contains Smith's second grade, then allgrades(2) is assigned the value 82. And so on.) Note that this assumes that the order of the grades is the same within each student.
* ("if last.idno then output;") If the input observation is the last observation in the data set that contains the student idno, then dump the program data vector (which contains allgrades) to the output data set. (For example, if the input observation is Smith's final exam grade, then output the now fat observation containing his six grades). If the input observation is not the last observation in the data set that contains the student idno, do nothing other than advance to the next step.
* ("i = i + 1;") Then, increase the index variable i by 1. (For example, if i is 1, change i to 2.)
* ("retain E1-E4 P1 F1 i;") Rather than setting E1, E2, E3, E4, P1, F1, and i to missing at the beginning of the next iteration of the data step, retain their current values. (So, for example, for Smith, allgrades(1) would retain its value of 78, allgrades (2) would retain its value of 82, and so on.) Similar for the index coutner i. We do not want to reset this variable until we get to a new student.

The program would keep cycling through the above five steps until it encountered the last observation in the data set. Then, the variables i, gtype, and grade would be dropped from the output fatgrades data set.  

### Example

SAS also has procedure called PROC TRANSPOSE that can be used to transpose datasets between wide and tall formats. Personally, I find this function somewhat unintuitive compared to the DATA step method (at least once you get used to using the DATA step), so I tend to always use the DATA step. However, in this next example we will show how to perform the same transposition using PROC TRANSPOSE and leave it to the reader to decide which method is preferred.  

In [58]:
PROC TRANSPOSE data = tallgrades 
               out = fatgrades2 (drop = _NAME_) ;
    BY idno l_name;
    VAR grade;
    ID gtype;
RUN;
 
PROC PRINT data = fatgrades2;
    title 'The fat grades data set';
RUN;

Obs,idno,l_name,E1,E2,E3,E4,P1,F1
1,10,Smith,78,82,86,69,97,160
2,11,Simon,88,72,86,99,100,170
3,12,Jones,98,92,92,99,99,185


In PROC TRANSPOSE,  
* The OUT statement, as with PROC SORT, specifies the name of the output dataset created by PROC TRANSPOSE.
* The BY clause specifies the variable(s) that are unique within the groups between transposed to a single row. In this case, a single observation corresponds to a given student. Although, idno is the main key, last name (l_name) should also be kept. Without adding it to the BY statement, l_name would be dropped from the transposed dataset.
* The VAR statement specifies the variable containing the value of the transposed variable. In this case, it is the actual grade value.
* The ID statement can be used to specify column names from an existing variable's values. In this case, we use the assessment name to identify the grade.

### 広い(横長の)データから長い(縦長の)データへの再形成

次の2つの例では、作成した成績データセットの広い版を元の縦長のデータセット版に変換する方法を学びます。

例
この例では、DATAステップを使って成績データセットを広い形式から縦長の形式に転置します。




### Reshaping Data from Wide (Fat) to Tall (Long)

In the next two examples, we will learn how to transform the newly created wide version of the grades dataset back to the tall dataset version.

### Example

In this example, we will use a DATA step to transpose the grades dataset from wide back to tall format.  

In [69]:
DATA tallgrades2;
  set fatgrades;
  array gtypes(6) $ _TEMPORARY_ ('E1' 'E2' 'E3' 'E4' 'P1' 'F1');
  array grades(*) E1 -- F1;
  DO i = 1 TO 6;
      gtype = gtypes(i);
      grade = grades(i);
      OUTPUT;
  END;
  DROP E1--F1 i ;
RUN;

PROC PRINT data = tallgrades2 NOOBS;
   TITLE 'Tallgrades2 Data';
RUN;

idno,l_name,gtype,grade
10,Smith,E1,78
10,Smith,E2,82
10,Smith,E3,86
10,Smith,E4,69
10,Smith,P1,97
10,Smith,F1,160
11,Simon,E1,88
11,Simon,E2,72
11,Simon,E3,86
11,Simon,E4,99


gtypesは一時的な文字列配列で、列名(評価タイプ)を変数gtypesに返し、gradesは現在の行(つまり現在の生徒)の成績を格納して、変数gradeに割り当てるためのものです。DOループの各反復にはOUTPUTステートメントがあり、各成績を独立した行に出力します。idnoとl_nameは、出力するべき成績が残っている限り、行から行へと引き継がれます。そして、DOループを抜けて次の行(生徒)に移ります。最後に、(横長の)列E1、E2、E3、E4、P1、F1とインデックスiを最終的なデータセットから削除して、元の縦長のデータセットを取得します。

列名を取得する別の方法は、文字列配列にそれらを手動でリストアップするのではなく、vnameという関数を使うことです。vname関数は、変数に適用すると、その変数名を文字列値として返します。



We create two arrays: gtypes is a temporary character array to return the columns names (the assessment types) to the variable gtypes and grades to store the grades on the current row (i.e. current student) to be assigned to the grade variable. Each iteration of the DO loop has an OUPUT statement to put each grade in its own row. Note that the idno and l_name are carried over from row to row until we run out of grades to OUTPUT. Then we exit the DO loop and move on the next row (student). We drop the (wide) columns E1, E2, E3, E4, P1, F1, and the index i from the final dataset to obtain the original tall dataset.  
An alternative way to get the column names is to use the **vname** function instead of manually listing out the names in an character array. The vname function when applied to a variable returns the variable name as a character value.  

In [15]:
DATA tallgrades3;
  set fatgrades;
  array grades(*) E1 -- F1;
  DO i = 1 TO 6;
      gtype = vname(grades(i));
      grade = grades(i);
      OUTPUT;
  END;
  DROP E1--F1 i ;
RUN;

PROC PRINT data = tallgrades3 NOOBS;
   TITLE 'Tallgrades3 Data';
RUN;

idno,l_name,gtype,grade
10,Smith,E1,78
10,Smith,E2,82
10,Smith,E3,86
10,Smith,E4,69
10,Smith,P1,97
10,Smith,F1,160
11,Simon,E1,88
11,Simon,E2,72
11,Simon,E3,86
11,Simon,E4,99


プロシージャ TRANSPOSE を使って同じ操作を行う方法も紹介します。




### Example

Now we will do the same operation as the previous example but using PROC TRANSPOSE.  

In [71]:
PROC TRANSPOSE data = fatgrades 
               out = tallgrades3 (RENAME = (COL1 = grade 
                                            _NAME_ = gtype));
   BY idno l_name;
   VAR E1--F1;
RUN;

PROC PRINT data = tallgrades3;
RUN;

Obs,idno,l_name,gtype,grade
1,10,Smith,E1,78
2,10,Smith,E2,82
3,10,Smith,E3,86
4,10,Smith,E4,69
5,10,Smith,P1,97
6,10,Smith,F1,160
7,11,Simon,E1,88
8,11,Simon,E2,72
9,11,Simon,E3,86
10,11,Simon,E4,99


BYステートメントは、単一の観測値を定義するグループ化変数を定義し、広い形式から長い形式に移行する際に、それらの変数を新しい行にコピーします。VARステートメントは、2つの新しい列 _NAME_ と _COL1_ を定義することで、複数行に集約される列をすべて定義します。_NAME_ には以前の列名が格納され、_COL1_ には現在の行のその以前の列からのデータ値が格納されます。通常、RENAME データセットオプションを使って、これらのデフォルト名を変更したいと思うでしょう。


The BY statement defines the grouping variables that define a single observation and are copied to each new row when going from wide to long. The VAR statement defines all the columns that should be gatherd into a multiple rows by defining two new columns _NAME_ which holds the former column name and _COL1_ which holds the data value from that former column in the current row. Typically, we will want to change these default names by using the RENAME dataset option.  

## データセットの結合

このセクションでは、以下の方法で複数のSASデータセットを単一のデータセットに結合する方法を学びます。

* SET文を使用して2つのデータセットを垂直に連結する
* MERGEステートメントを使用してデータセットをマージし、内部結合、外部結合、左結合、右結合を行う

### 2つ以上のデータセットの連結

2つ以上のSASデータセットを連結するとは、単一のSASデータセットに1つの"上に"もう1つを積み重ねることを意味します。例えば、データセットstore1には、`store`(番号)、`day`(曜日)、`sales`(ドル)の3つの変数が含まれているとします。

連結すると、データセットが上に積み重なった形になります。新しいデータセットの観測値の数は、元のデータセットの観測値の数の合計になることに注意してください。SASデータセットを連結するには、単にSET文で一覧のデータセット名を指定するだけです。

変数の属性には、変数の種類(文字型か数値型か)、インフォーマット(変数の読み込み方法)とフォーマット(値の出力方法)、変数の長さ、ラベル(変数名の出力方法)などがあります。入力データセット間で変数属性が異なる場合、連結にはいくつかの問題があります。

* SET文で名前を指定したデータセットに、名前と種類が同じ変数が含まれている場合は、データセットを変更せずに連結できます。
* 変数の種類が異なる場合は、連結する前に1つ以上のデータセットを修正する必要があります。SASはデータセットを連結しません。
* 長さ、フォーマット、インフォーマット、ラベルが異なる場合は、連結する前に1つ以上のデータセットを修正したい場合があります。SASはデータセットを連結しますが、結果が気に入らない可能性があります。これらの属性は、(SET文の)最初に読み込まれたデータセットから取得されます。



## Merging Datasets

In this section, we will learn how to combine multiple SAS datasets into a single dataset by

* Concatenating two datatsets vertically with the SET statement
* Merging datasets with the MERGE statement to perform inner, outer, left and right joins.

### Concatenating Two or More Datasets

To concatenate two or more SAS data sets means to stack one "on top" of the other into a single SAS data set. For example, suppose the data set store1 contains three variables, `store` (number), `day` (of the week), and `sales` (in dollars):

|Store|Day|Sale|
|-----|---|----|
|1| M| 1200|
|1| T| 1435|
|1| W| 1712|
|1| R| 1529|
|1| F| 1920|
|1| S| 2325|

and the data set store2 contains the same three variables:

|Store|Day|Sale|
|-----|---|----|
|2| M| 2215|
|2| T| 2458|
|2| W| 1789|
|2| R| 1692|
|2| F| 2105|
|2| S| 2847|

Then, when we concatenate the two data sets, we get what I like to call a "tall" data set:

|Store|Day|Sale|
|-----|---|----|
|1| M| 1200|
|1| T| 1435|
|1| W| 1712|
|1| R| 1529|
|1| F| 1920|
|1| S| 2325|
|2| M| 2215|
|2| T| 2458|
|2| W| 1789|
|2| R| 1692|
|2| F| 2105|
|2| S| 2847|

in which the data sets are stacked on top of each other. Note that the number of observations in the new data set is the sum of the numbers of observations in the original data sets. To concatenate SAS data sets, you simplify specify a list of data set names in one SET statement.

As you know, variable attributes include the type of variable (character vs. numeric), the informat (how the variable is read in) and format (how its values are printed) of a variable, the length of the variable, and the label (how its variable name is printed) of a variable. Concatenating data sets when variable attributes differ across the input data sets may pose problems for SAS (and therefore you):

* If the data sets you name in the SET statement contain variables with the same names and types, you can concatenate the data sets without modification.
* If the variable types differ, you must modify one or more of the data sets before concatenating them. SAS will not concatenate the data sets until you do.
* If the lengths, formats, informats or labels differ, you may want to modify one or more of the data sets before concatenating them. SAS will concatenate the data sets; you may just not like the results. These attributes will be taken from the dataset that is read in first (i.e. appears first in the SET statement).

### Example

The following program concatenates the store1 and store2 data sets to create a new "tall" data set called bothstores:  

In [2]:
DATA store1;
    input Store Day $ Sales;
    DATALINES;
    1 M 1200
    1 T 1435
    1 W 1712
    1 R 1529
    1 F 1920
    1 S 2325
    ;
RUN;
 
DATA store2;
    input Store Day $ Sales;
    DATALINES;
    2 M 2215
    2 T 2458
    2 W 1798
    2 R 1692
    2 F 2105
    2 S 2847
    ;
RUN;
 
DATA bothstores;
    set store1 store2;
RUN;
 
PROC PRINT data = bothstores NOOBS;
    title 'The bothstores data set';
RUN;

Store,Day,Sales
1,M,1200
1,T,1435
1,W,1712
1,R,1529
1,F,1920
1,S,2325
2,M,2215
2,T,2458
2,W,1798
2,R,1692


Note that the input data sets — store1 and store2 — contain the same variables — Store, Day, and Sales — with identical attributes. In the third DATA step, the DATA statement tells SAS to create a new data set called bothstores, and the SET statement tells SAS that the data set should contain first the observations from store1 and then the observations from store2. Note that although we have specified only two input data sets here, the SET statement can contain any number of input data sets.  
Launch and run the SAS program, and review the output from the PRINT procedure to convince yourself that SAS did indeed concatenate the store1 and store2 data sets to make one "tall" data set called bothstores. You might then want to edit the SET statement so that store1 follows store2, and re-run the SAS program to see that then the contents of store1 follow the contents of store2 in the bothstores data set.  
In general, a data set that is created by concatenating data sets contains all of the variables and all of the observations from all of the input data sets. Therefore, the number of variables the new data set contains always equals the total number of unique variables among all of the input data sets. And, the number of observations in the new data set is the sum of the numbers of observations in the input data sets. Let's return to the contrived example we've used throughout this lesson.  

### Example

The following program concatenates the one and two data sets to create a new "tall" data set called onetopstwo:  

In [3]:
DATA one;
    input ID VarA $ VarB $;
    DATALINES;
    10 A1 B1
    20 A2 B2
    30 A3 B3
    ;
RUN;
 
DATA two;
    input ID VarB $ VarC $;
    DATALINES;
    40 B4 C1
    50 B5 C2
    ;
RUN;
 
DATA onetopstwo;
    set one two;
RUN;
 
PROC PRINT data = onetopstwo NOOBS;
    title 'The onetopstwo data set';
RUN;

ID,VarA,VarB,VarC
10,A1,B1,
20,A2,B2,
30,A3,B3,
40,,B4,C1
50,,B5,C2


As you review the first two DATA steps, in which SAS reads in the respective one and two data sets, note that the total number of unique variables is four — ID, VarA, VarB, and VarC. The total number of observations among the two input data sets is 3 + 2 = 5. Therefore, we can expect the concatenated data set onetopstwo to contain four variables and five observations. Launch and run the SAS program, and review the output to convince yourself that SAS did grab first all of the variables and all of the observations from the one data set and then all of the variables and all of the observations from the two data set. As you can see, to make it all work out okay, observations arising from the one data set have missing values for VarC, and observations from the two data set have missing values for VarA.  

### データセットのマッチマージ

マッチマージは、2つ以上のSASデータセットを結合する最も強力な方法の1つです。マッチマージでは、1つ以上の共通変数の値に基づいて、データセット間で観測値を組み合わせます。

例えば、`base`というフィクションのデータセットには、id 1〜10の患者のベースラインデータと年齢が含まれており、`visits`データセットには、id 1〜8と11の患者のデータ、訪問番号(すべての患者で3回の訪問)、および定量的な結果測定値が含まれているとします。

マッチマージを行うには、マージするデータセットをMERGEステートメントで指定し、マージ対象の変数をBYステートメントで示します。ただし、**マージする変数(BYステートメントに現れる変数)でソートされていないと、SASデータセットをマッチマージできない**ことに注意が必要です。マージ対象の変数は、両方のデータセットで同じ名前でなければなりません。そうでない場合は、マージする前に、renameオプションを使って一方のデータセットの変数名を変更する必要があります。




### Match-Merging SAS Datasets

Match-merging is one of the most powerful methods of combining two or more SAS data sets. A match-merge combines observations across data sets based on the values of one or more common variables. 

Consider a fictional dataset called `base` that has baseline data for patients with the ids 1 to 10 and their age, and  `visits` dataset which contains data for patients with the ids 1 to 8 and 11, the visit number (all patients have 3 visits), and some quantitative outcome measurement.

To match-merge, you simply specify the data sets you would like to merge in a MERGE statement, and indicate the variables on which you would like to merge in a BY statement. One thing to keep in mind, though, is **you can't match-merge SAS data sets unless they are sorted by the variables appearing in the BY statement**. The variables that you are merging on (i.e that appear in the BY statement) must have the same name in both datasets. If this is not the case, then use the rename option to change the variable name in one of the datasets to match the other before merging.

### Example

In the following SAS program, we will perform an outer join of the base and visits dataset by merging based in the patient id variable.  

In [4]:
DATA base;
   INPUT id age;
   DATALINES;
1 50
2 51
3 52
4 53
5 54
6 55
7 56
8 57
9 58
10 59
;
RUN;

DATA visits;
    DO id = 1 TO 8;
       DO visit = 1 TO 3;
           outcome = 10*id + visit;
           OUTPUT;
        END;
    END;
    id = 11;
    visit = 3;
    outcome = 50;
    OUTPUT;
RUN;

PROC PRINT data = base;
   TITLE "Baseline Dataset";
RUN;

PROC PRINT data = visits;
   TITLE "Visits Dataset";
RUN;

Obs,id,age
1,1,50
2,2,51
3,3,52
4,4,53
5,5,54
6,6,55
7,7,56
8,8,57
9,9,58
10,10,59


To peform an outer join between the base and visits dataset, we simply use the MERGE statemet with these two datasets and a BY statement with the common variable id. Note that these two datasets are already sorted by id. If they were not, we would also need to sort both datasets by id first.  

In [5]:
DATA outer;
   MERGE base visits;
   BY id;
RUN;

PROC PRINT data = outer;
   TITLE "Outer Join";
RUN;

Obs,id,age,visit,outcome
1,1,50,1,11
2,1,50,2,12
3,1,50,3,13
4,2,51,1,21
5,2,51,2,22
6,2,51,3,23
7,3,52,1,31
8,3,52,2,32
9,3,52,3,33
10,4,53,1,41


In this merge, all variables and rows from both datasets are kept with the id columns from both datasets merged to a single column. Note that for id's that appeared in base and not in visits, the variables in visits, visit and outcome, that were not in base were set to missing (see id 9 and 10), and for ids in the visit dataset that were not in the base dataset, the variables that were in base but not in visit, age in this case, were set to missing (see id 11).  

Before we learn how to do inner, left and right joins, we need to discuss the IN= SAS dataset option. The IN= option tells SAS to create an "indicator variable" that takes on either the value 0 or 1 depending on whether or not the current observation comes from the input data set. If the observation does not come from the input data set, then the indicator variable takes on the value 0. If the observation does come from the input data set, then the indicator takes on the value 1. The IN= option is especially useful when merging and concatenating data sets which we'll study in the next two lessons. The basic format of the IN option is:

`IN = varname`

where varname is the name of a variable in the input data set. Let's revisit the previous example where we keep track of the IN= variable values.

### Example

Let's merge base and visit again, but this time create the IN= variables for each dataset and store their values in two permanent variables in_base and in_visit:  

In [7]:
DATA outer2;
   MERGE base (IN = in1) 
         visits (IN = in2);
   BY id;
   in_base = in1;
   in_visit = in2;
RUN;

PROC PRINT data = outer2 (FIRSTOBS=22 OBS=27);
   TITLE "Outer Join with IN Variables";
RUN;

Obs,id,age,visit,outcome,in_base,in_visit
22,8,57,1,81,1,1
23,8,57,2,82,1,1
24,8,57,3,83,1,1
25,9,58,.,.,1,0
26,10,59,.,.,1,0
27,11,.,3,50,0,1


The ids for patients 1 through 8 appear in both dataset, so for all of the rows corresponding to these ids both in_base and in_visit have the value 1 since we combined data from both datasets to create these rows. But for ids 9 and 10, these only appeared in the base dataset so in_base is 1 and in_visit is 0, since there was no data from visit to use to make these rows. Similarly for id 11, the only data was contained in the visit dataset so in_visit is 1 and in_base is 0.  

To perform an inner join, we only want to keep rows that have matching ids in both datasets. Using the IN= variables, we only want to keep rows in which both (or all if we have more than 2 datasets) are 1.

### Example

The following SAS program performs an inner join between the base and visit dataset. This means that we will only keep rows in which the merging variable, id, appear in both datasets.  

In [9]:
DATA inner;
   MERGE base (IN = in1) 
         visits (IN = in2);
   BY id;
   IF in1 = 1 AND in2 = 1; *subsetting IF. Only keep these rows;
RUN;

PROC PRINT data = inner;
   TITLE "Inner Join";
RUN;

Obs,id,age,visit,outcome
1,1,50,1,11
2,1,50,2,12
3,1,50,3,13
4,2,51,1,21
5,2,51,2,22
6,2,51,3,23
7,3,52,1,31
8,3,52,2,32
9,3,52,3,33
10,4,53,1,41


Now only the rows made from ids 1 to 8 are in the merged dataset because these were the only ids that occurred in both the base and visit datasets.  

### Example

In this example, we will perform left and right joins using the IN= variables. A left join will keep all records from the left (i.e. first dataset in the MERGE statement) and join matching records from the right dataset and dropping the rest. The reverse happesn for a right join.  

In [10]:
DATA left;
   MERGE base (IN = in1) 
         visits (IN = in2);
   BY id;
   IF in1 = 1; *subsetting IF. Only keep these rows;
RUN;

PROC PRINT data = left;
   TITLE "Left Join";
RUN;

DATA right;
   MERGE base (IN = in1) 
         visits (IN = in2);
   BY id;
   IF in2 = 1; *subsetting IF. Only keep these rows;
RUN;

PROC PRINT data = right;
   TITLE "Right Join";
RUN;

Obs,id,age,visit,outcome
1,1,50,1,11
2,1,50,2,12
3,1,50,3,13
4,2,51,1,21
5,2,51,2,22
6,2,51,3,23
7,3,52,1,31
8,3,52,2,32
9,3,52,3,33
10,4,53,1,41



## 演習

これらの演習では、Bike_Lanes_Wide.csv、Bike_Lanes.csv、crashes.csv、roads.csvを使用します。これらのデータセットがコンピューター上の便利な場所にダウンロードされていることを確認してください。コースウェブページにあるdatasetsフォルダを参照してください。

1. Bike_Lanes_Wide.csvデータセットをPROC IMPORTで読み込み、`wide`と呼びます。データセットの最初の数行を出力します。
2. `wide`をPROC TRANSPOSEまたはDATAステップを使って長い形式に変形します。nameカラム以外のすべてのカラムを集める必要があります。新しい2つのカラム `lanetype`(元のカラム名)と`the_length`(データ値)を持つ長いデータセットに変換します。変数`the_length`の'NA'の値を.に置き換え、数値カラムに変換します。
3. roads.csvとcrashes.csvファイルを読み込み、`road`と`crash`と呼びます。
4. `crash`の`Road`変数にあるハイフン(`-`)をすべて空白に置き換えます(`tranwrd`を使用)。この結果を`crash2`と呼びます。`Road`変数をPROC FREQでテーブル化します。
5. `crash`と`road`のそれぞれのデータセットに何件の観測値がありますか?
6. `crash2`の`Road`カラムを`scan`を使って(`type`と`number`)に分割します。この結果を再び`crash2`に割り当てます。`crash2`の`type`をPROC FREQでテーブル化します。次に、連結関数(CATなど)を使って`road_hyphen`という新しい変数を作成します。`type`と`number`の列をハイフン(`-`)で結合し、`road_hyphen`をPROC FREQでテーブル化します。
7. `crash`データセットでは、どの年のデータが収集されましたか?何年分ありますか?
8. Bike_Lanes.csvデータセットを読み込み、`bike`と呼びます。
9. `type`と`name`に欠損値がない行のみを残し、出力を`bike`に再割り当てします。
10. PROC MEANSでBYステートメントを使い、`name`と`type`でグループ化(各nameごとのtypeごとの意味)し、`length`の`sum`を求めます。OUTPUTステートメントを使ってこの要約データセットを取得し、`name`、`type`、`length`の`sum`のカラム(これを`length`に名前を変更)のみを保持します。このデータセットを`sub`と呼びます。
11. `sub`を長い形式

## Exercises

For these exercises, we will use the Bike_Lanes_Wide.csv, Bike_Lanes.csv, crashes.csv and roads.csv. Please be sure that you have these datasets downloaded to a convenient location on your computer. See the datasets folder provided on the course webpage.

1. Read in the Bike_Lanes_Wide.csv dataset and call is `wide` using PROC IMPORT. Print the first few rows of the dataset.
2. Reshape `wide` using either PROC TRANSPOSE or a DATA step. You will need to gather all columns except the name column. Transform into a long dataset with two new columns `lanetype` (the former column names) and `the_length` the data values. In the variable `the_length`, replace 'NA' values with . and convert it to a numeric column.
3. Read in the roads and crashes .csv files and call them `road` and `crash`.
4. Replace (using `tranwrd`) any hyphens (`-`) with a space in `Road` variable of `crash`. Call this data `crash2`. Table the `Road` variable with PROC FREQ.
5. How many observations are in each of the `crash` and `road` datasets?
6. Separate the `Road` column (using `scan`) into (`type` and `number`) in `crash2`. Reassign this to `crash2`. Table `type` from `crash2` using PROC FREQ. Then create a new variable calling it `road_hyphen` using one of the concatenate functions (such as CAT). Unite the `type` and `number` columns using a hyphen (`-`) and then table `road_hyphen` using PROC FREQ.
7. Which and how many years were data collected in the `crash` dataset?
8. Read in the dataset Bike_Lanes.csv and call it `bike`.
9. Keep rows where the record is not missing `type` and not missing `name` and re-assign the output to `bike`.
10. Using PROC MEANS with a BY statement grouping `name` and `type` (i.e for each type within each name), find the `sum` of the `length`. Use an OUTPUT statement to get this summary dataset and only keep the `name`, `type` and sum of the `length` column (after renaming this column `length`). Call this data set `sub`.
11. Reshape `sub` from long to wide by taking the `type` to be the new columns and `length` to be the value in those columns. (NOTE: the `names` have spaces in them. Do we need to replace the spaces with a character before changing them to column names? The DATA step approach is more challenging here since not all streets have the same bike lane types.)
12. Join data in the crash and road datasets to retain only complete data, (using an inner join on road) Merge by the variable `Road`. Call the output merged. How many observations are there?
13. Join data using a `full_join`. Call the output `full`. How many observations are there?
14. Do a left join of the `road` and `crash`. ORDER matters here! How many observations are there?
15. Repeat above with a `right_join` with the same order of the arguments. How many observations are there?